In [1]:
"""
Imports und paths
"""

import os
import copy
import time
import random
import math
import itertools
import numpy as np
import tensorflow as tf

model_path = os.environ.get('MODEL_PATH', 'models/')
summary_path = os.environ.get('SUMMARY_PATH', 'summaries/')
checkpoint_path = os.environ.get('CHECKPOINT_PATH', 'checkpoints/')
own_checkpoint_path = os.environ.get('CHECKPOINT_PATH', 'own_checkpoints/')

In [3]:
"""
Ursprünglich von https://github.com/awni/backgammon kopiert und dann abgeändert.
"""

class Game:

    LAYOUT = "0-2-o,5-5-x,7-3-x,11-5-o,12-5-x,16-3-o,18-5-o,23-2-x"
    NUMCOLS = 24
    QUAD = 6
    OFF = 'off'
    ON = 'on'
    TOKENS = ['x', 'o']

    def __init__(self, layout=LAYOUT, grid=None, off_pieces=None, bar_pieces=None, num_pieces=None, players=None):
        """
        Define a new game object
        """
        self.die = Game.QUAD
        self.layout = layout
        if grid:
            self.grid = copy.deepcopy(grid)
            self.off_pieces = copy.deepcopy(off_pieces)
            self.bar_pieces = copy.deepcopy(bar_pieces)
            self.num_pieces = copy.deepcopy(num_pieces)
            self.players = players
            return
        self.players = Game.TOKENS
        self.grid = [[] for _ in range(Game.NUMCOLS)]
        self.off_pieces = {}
        self.bar_pieces = {}
        self.num_pieces = {}
        for t in self.players:
            self.bar_pieces[t] = []
            self.off_pieces[t] = []
            self.num_pieces[t] = 0

    @staticmethod
    def new():
        game = Game()
        game.reset()
        return game

    def extract_features(self, player):
        features = []
        for p in self.players:
            for col in self.grid:
                feats = [0.] * 6
                if len(col) > 0 and col[0] == p:
                    for i in range(len(col)):
                        feats[min(i, 5)] += 1
                features += feats
            features.append(float(len(self.bar_pieces[p])) / 2.)
            features.append(float(len(self.off_pieces[p])) / self.num_pieces[p])
        if player == self.players[0]:
            features += [1., 0.]
        else:
            features += [0., 1.]
        return np.array(features).reshape(1, -1)
    
    # Methode die exakt die 198 Features liefert die in TD-Gammon 0.0 benutzt wurden
    # Nach "Reinforcement Learning: An Introduction", Sutton & Barto, 2017
    def extractFeatures(self, player):
        features = []
        # 196 Features kodieren den Zustand der Spielfelder, 98 für jeden Spieler
        for p in self.players:
            # 24 mögliche Brettpositionen
            for col in self.grid:
                # 4 Features kodieren eine Stelle auf dem Spielbrett
                feats = [0.] * 4
                if len(col) > 0 and col[0] == p:
                    # 0,1,2,3,4,5 Steine werden kodiert als
                    # 0000, 1000, 1100, 1110, 1110.5, 1111
                    # (4. Bit = (n-3)/2)
                    for i in range(len(col)):
                        if i < 3:
                            feats[i] += 1
                        else:
                            feats[3] = (len(col)-3)/2.0
                            break
                features += feats
            # Anzahl der Steine auf der "Bar", n/2
            features.append(float(len(self.bar_pieces[p])) / 2.)
            # Anzahl der Steine die bereits aus dem Spiel sind, n/15
            features.append(float(len(self.off_pieces[p])) / 15.)
        # Zwei Features für den derzeitigen Spieler
        if player == self.players[0]:
            features += [1., 0.]
        else:
            features += [0., 1.]
        return np.array(features).reshape(1, -1)

    def roll_dice(self):
        return (random.randint(1, self.die), random.randint(1, self.die))

    def play(self, players, draw=False):
        player_num = random.randint(0, 1)
        while not self.is_over():
            self.next_step(players[player_num], player_num, draw=draw)
            player_num = (player_num + 1) % 2
        return self.winner()

    def next_step(self, player, player_num, draw=False):
        roll = self.roll_dice()

        if draw:
            self.draw()

        self.take_turn(player, roll, draw=draw)

    def take_turn(self, player, roll, draw=False):
        if draw:
            print("Player %s rolled <%d, %d>." % (player.player, roll[0], roll[1]))
            time.sleep(1)

        moves = self.get_actions(roll, player.player, nodups=True)
        move = player.get_action(moves, self) if moves else None

        if move:
            self.take_action(move, player.player)

    def clone(self):
        """
        Return an exact copy of the game. Changes can be made
        to the cloned version without affecting the original.
        """
        return Game(None, self.grid, self.off_pieces,
                    self.bar_pieces, self.num_pieces, self.players)

    def take_action(self, action, token):
        """
        Makes given move for player, assumes move is valid,
        will remove pieces from play
        """
        ateList = [0] * 4
        for i, (s, e) in enumerate(action):
            if s == Game.ON:
                piece = self.bar_pieces[token].pop()
            else:
                piece = self.grid[s].pop()
            if e == Game.OFF:
                self.off_pieces[token].append(piece)
                continue
            if len(self.grid[e]) > 0 and self.grid[e][0] != token:
                bar_piece = self.grid[e].pop()
                self.bar_pieces[bar_piece].append(bar_piece)
                ateList[i] = 1
            self.grid[e].append(piece)
        return ateList

    def undo_action(self, action, player, ateList):
        """
        Reverses given move for player, assumes move is valid,
        will remove pieces from play
        """
        for i, (s, e) in enumerate(reversed(action)):
            if e == Game.OFF:
                piece = self.off_pieces[player].pop()
            else:
                piece = self.grid[e].pop()
                if ateList[len(action) - 1 - i]:
                    bar_piece = self.bar_pieces[self.opponent(player)].pop()
                    self.grid[e].append(bar_piece)
            if s == Game.ON:
                self.bar_pieces[player].append(piece)
            else:
                self.grid[s].append(piece)


    def get_actions(self, roll, player, nodups=False):
        """
        Get set of all possible move tuples
        """
        moves = set()
        if nodups:
            start = 0
        else:
            start = None

        r1, r2 = roll
        if r1 == r2: # doubles
            i = 4
            # keep trying until we find some moves
            while not moves and i > 0:
                self.find_moves(tuple([r1]*i), player, (), moves, start)
                i -= 1
        else:
            self.find_moves(roll, player, (), moves, start)
            self.find_moves((r2, r1), player, (), moves, start)
            # has no moves, try moving only one piece
            if not moves:
                for r in roll:
                    self.find_moves((r, ), player, (), moves, start)

        return moves

    def find_moves(self, rs, player, move, moves, start=None):
        if len(rs)==0:
            moves.add(move)
            return
        r, rs = rs[0], rs[1:]
        # see if we can remove a piece from the bar
        if self.bar_pieces[player]:
            if self.can_onboard(player, r):
                piece = self.bar_pieces[player].pop()
                bar_piece = None
                if len(self.grid[r - 1]) == 1 and self.grid[r - 1][-1]!=player:
                    bar_piece = self.grid[r - 1].pop()

                self.grid[r - 1].append(piece)

                self.find_moves(rs, player, move+((Game.ON, r - 1), ), moves, start)
                self.grid[r - 1].pop()
                self.bar_pieces[player].append(piece)
                if bar_piece:
                    self.grid[r - 1].append(bar_piece)
            return

        # otherwise check each grid location for valid move using r
        offboarding = self.can_offboard(player)

        for i in range(len(self.grid)):
            if start is not None:
                start = i
            if self.is_valid_move(i, i + r, player):

                piece = self.grid[i].pop()
                bar_piece = None
                if len(self.grid[i+r]) == 1 and self.grid[i+r][-1] != player:
                    bar_piece = self.grid[i + r].pop()
                self.grid[i + r].append(piece)
                self.find_moves(rs, player, move + ((i, i + r), ), moves, start)
                self.grid[i + r].pop()
                self.grid[i].append(piece)
                if bar_piece:
                    self.grid[i + r].append(bar_piece)

            # If we can't move on the board can we take the piece off?
            if offboarding and self.remove_piece(player, i, r):
                piece = self.grid[i].pop()
                self.off_pieces[player].append(piece)
                self.find_moves(rs, player, move + ((i, Game.OFF), ), moves, start)
                self.off_pieces[player].pop()
                self.grid[i].append(piece)

    def opponent(self, token):
        """
        Retrieve opponent players token for a given players token.
        """
        for t in self.players:
            if t != token:
                return t

    def is_won(self, player):
        """
        If game is over and player won, return True, else return False
        """
        return self.is_over() and player == self.players[self.winner()]

    def is_lost(self, player):
        """
        If game is over and player lost, return True, else return False
        """
        return self.is_over() and player != self.players[self.winner()]

    def reverse(self):
        """
        Reverses a game allowing it to be seen by the opponent
        from the same perspective
        """
        self.grid.reverse()
        self.players.reverse()

    def reset(self):
        """
        Resets game to original layout.
        """
        for col in self.layout.split(','):
            loc, num, token = col.split('-')
            self.grid[int(loc)] = [token for _ in range(int(num))]
        for col in self.grid:
            for piece in col:
                self.num_pieces[piece] += 1

    def winner(self):
        """
        Get winner.
        """
        return 0 if len(self.off_pieces[self.players[0]]) == self.num_pieces[self.players[0]] else 1

    def is_over(self):
        """
        Checks if the game is over.
        """
        for t in self.players:
            if len(self.off_pieces[t]) == self.num_pieces[t]:
                return True
        return False

    def can_offboard(self, player):
        count = 0
        for i in range(Game.NUMCOLS - self.die, Game.NUMCOLS):
            if len(self.grid[i]) > 0 and self.grid[i][0] == player:
                count += len(self.grid[i])
        if count+len(self.off_pieces[player]) == self.num_pieces[player]:
            return True
        return False

    def can_onboard(self, player, r):
        """
        Can we take a players piece on the bar to a position
        on the grid given by roll-1?
        """
        if len(self.grid[r - 1]) <= 1 or self.grid[r - 1][0] == player:
            return True
        else:
            return False

    def remove_piece(self, player, start, r):
        """
        Can we remove a piece from location start with roll r ?
        In this function we assume we are cool to offboard,
        i.e. no pieces on the bar and all are in the home quadrant.
        """
        if start < Game.NUMCOLS - self.die:
            return False
        if len(self.grid[start]) == 0 or self.grid[start][0] != player:
            return False
        if start + r == Game.NUMCOLS:
            return True
        if start + r > Game.NUMCOLS:
            for i in range(start - 1, Game.NUMCOLS - self.die - 1, -1):
                if len(self.grid[i]) != 0 and self.grid[i][0] == self.players[0]:
                    return False
            return True
        return False

    def is_valid_move(self, start, end, token):
        if len(self.grid[start]) > 0 and self.grid[start][0] == token:
            if end < 0 or end >= len(self.grid):
                return False
            if len(self.grid[end]) <= 1:
                return True
            if len(self.grid[end]) > 1 and self.grid[end][-1] == token:
                return True
        return False

    def draw_col(self,i,col):
        print("|", end = '')
        if i==-2:
            if col<10:
                print(" ", end = '')
            print(str(col), end = '')
        elif i==-1:
            print("--", end = '')
        elif len(self.grid[col])>i:
            print(" "+self.grid[col][i], end = '')
        else:
            print("  ", end = '')

    def draw(self):
        largest = max([len(self.grid[i]) for i in range(int(len(self.grid)/2),len(self.grid))])
        for i in range(-2,largest):
            for col in range(int(len(self.grid)/2),len(self.grid)):
                self.draw_col(i,col)
            print("|")
        print()
        print()
        largest = max([len(self.grid[i]) for i in range(int(len(self.grid)/2))])
        for i in range(largest-1,-3,-1):
            for col in range(int(len(self.grid)/2)-1,-1,-1):
                self.draw_col(i,col)
            print("|")
        for t in self.players:
            print("<Player %s>  Off Board : "%(t), end = '')
            for piece in self.off_pieces[t]:
                print(t+'', end = '')
            print("   Bar : ", end = '')
            for piece in self.bar_pieces[t]:
                print(t+'', end = '')
            print()

In [3]:
#Debug testcode
game = Game.new()
game.draw()
x = game.extractFeatures(Game.TOKENS[0])
print(x)
print(x.shape)

|12|13|14|15|16|17|18|19|20|21|22|23|
|--|--|--|--|--|--|--|--|--|--|--|--|
| x|  |  |  | o|  | o|  |  |  |  | x|
| x|  |  |  | o|  | o|  |  |  |  | x|
| x|  |  |  | o|  | o|  |  |  |  |  |
| x|  |  |  |  |  | o|  |  |  |  |  |
| x|  |  |  |  |  | o|  |  |  |  |  |


| o|  |  |  |  |  | x|  |  |  |  |  |
| o|  |  |  |  |  | x|  |  |  |  |  |
| o|  |  |  | x|  | x|  |  |  |  |  |
| o|  |  |  | x|  | x|  |  |  |  | o|
| o|  |  |  | x|  | x|  |  |  |  | o|
|--|--|--|--|--|--|--|--|--|--|--|--|
|11|10| 9| 8| 7| 6| 5| 4| 3| 2| 1| 0|
<Player x>  Off Board :    Bar : 
<Player o>  Off Board :    Bar : 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
  0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.
  0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.

In [12]:
#Random vs Random alte Engine
game = Game.new()
game.play([RandomAgent(Game.TOKENS[0]), RandomAgent(Game.TOKENS[1])], draw=True)

|12|13|14|15|16|17|18|19|20|21|22|23|
|--|--|--|--|--|--|--|--|--|--|--|--|
| x|  |  |  | o|  | o|  |  |  |  | x|
| x|  |  |  | o|  | o|  |  |  |  | x|
| x|  |  |  | o|  | o|  |  |  |  |  |
| x|  |  |  |  |  | o|  |  |  |  |  |
| x|  |  |  |  |  | o|  |  |  |  |  |


| o|  |  |  |  |  | x|  |  |  |  |  |
| o|  |  |  |  |  | x|  |  |  |  |  |
| o|  |  |  | x|  | x|  |  |  |  |  |
| o|  |  |  | x|  | x|  |  |  |  | o|
| o|  |  |  | x|  | x|  |  |  |  | o|
|--|--|--|--|--|--|--|--|--|--|--|--|
|11|10| 9| 8| 7| 6| 5| 4| 3| 2| 1| 0|
<Player x>  Off Board :    Bar : 
<Player o>  Off Board :    Bar : 
Player x rolled <3, 1>.
|12|13|14|15|16|17|18|19|20|21|22|23|
|--|--|--|--|--|--|--|--|--|--|--|--|
| x| x|  |  | o|  | o|  |  |  |  | x|
| x|  |  |  | o|  | o|  |  |  |  | x|
| x|  |  |  | o|  | o|  |  |  |  |  |
| x|  |  |  |  |  | o|  |  |  |  |  |
|  |  |  |  |  |  | o|  |  |  |  |  |


| o|  |  |  |  |  | x|  |  |  |  |  |
| o|  |  |  |  |  | x|  |  |  |  |  |
| o|  |  |  |  |  | x|  |  |  

|12|13|14|15|16|17|18|19|20|21|22|23|
|--|--|--|--|--|--|--|--|--|--|--|--|
| x| x| x|  |  | o| o| o| o| o| o| x|
| x| x|  |  |  |  | o|  |  | o| o| x|
| x|  |  |  |  |  | o|  |  | o|  |  |
| x|  |  |  |  |  |  |  |  |  |  |  |
| x|  |  |  |  |  |  |  |  |  |  |  |


|  | x|  |  |  |  |  |  |  |  |  |  |
| o| x|  |  |  |  |  |  |  |  |  |  |
| o| x|  | o|  |  | x|  |  |  | x| o|
|--|--|--|--|--|--|--|--|--|--|--|--|
|11|10| 9| 8| 7| 6| 5| 4| 3| 2| 1| 0|
<Player x>  Off Board :    Bar : 
<Player o>  Off Board :    Bar : 
Player o rolled <1, 4>.
|12|13|14|15|16|17|18|19|20|21|22|23|
|--|--|--|--|--|--|--|--|--|--|--|--|
| x| x| x|  |  |  | o| o| o| o| o| x|
| x| x|  |  |  |  | o| o|  | o| o| x|
| x|  |  |  |  |  |  |  |  | o|  |  |
| x|  |  |  |  |  |  |  |  | o|  |  |
| x|  |  |  |  |  |  |  |  |  |  |  |


|  | x|  |  |  |  |  |  |  |  |  |  |
| o| x|  |  |  |  |  |  |  |  |  |  |
| o| x|  | o|  |  | x|  |  |  | x| o|
|--|--|--|--|--|--|--|--|--|--|--|--|
|11|10| 9| 8| 7| 6| 5| 4| 3| 2

|12|13|14|15|16|17|18|19|20|21|22|23|
|--|--|--|--|--|--|--|--|--|--|--|--|
| x|  |  |  | x| x| o| o|  | o| o| x|
| x|  |  |  | x| x| o| o|  | o|  | x|
| x|  |  |  |  |  |  |  |  | o|  | x|
|  |  |  |  |  |  |  |  |  | o|  |  |
|  |  |  |  |  |  |  |  |  | o|  |  |
|  |  |  |  |  |  |  |  |  | o|  |  |
|  |  |  |  |  |  |  |  |  | o|  |  |
|  |  |  |  |  |  |  |  |  | o|  |  |


|  | x| x|  | x| x|  |  |  |  | x|  |
|--|--|--|--|--|--|--|--|--|--|--|--|
|11|10| 9| 8| 7| 6| 5| 4| 3| 2| 1| 0|
<Player x>  Off Board :    Bar : 
<Player o>  Off Board : oo   Bar : 
Player o rolled <2, 6>.
|12|13|14|15|16|17|18|19|20|21|22|23|
|--|--|--|--|--|--|--|--|--|--|--|--|
| x|  |  |  | x| x| o| o|  | o|  | x|
| x|  |  |  | x| x| o| o|  | o|  | x|
| x|  |  |  |  |  |  |  |  | o|  | x|
|  |  |  |  |  |  |  |  |  | o|  |  |
|  |  |  |  |  |  |  |  |  | o|  |  |
|  |  |  |  |  |  |  |  |  | o|  |  |
|  |  |  |  |  |  |  |  |  | o|  |  |


|  | x| x|  | x| x|  |  |  |  | x|  |
|--|--|--|--|--|--|--|--|--|

|12|13|14|15|16|17|18|19|20|21|22|23|
|--|--|--|--|--|--|--|--|--|--|--|--|
| x|  |  |  |  |  | x| x| x|  | o| x|
|  |  |  |  |  |  | x|  |  |  | o| x|
|  |  |  |  |  |  | x|  |  |  |  | x|
|  |  |  |  |  |  | x|  |  |  |  | x|
|  |  |  |  |  |  |  |  |  |  |  | x|
|  |  |  |  |  |  |  |  |  |  |  | x|


|  | x|  | x|  |  |  |  |  |  |  |  |
|--|--|--|--|--|--|--|--|--|--|--|--|
|11|10| 9| 8| 7| 6| 5| 4| 3| 2| 1| 0|
<Player x>  Off Board :    Bar : 
<Player o>  Off Board : ooooooooooooo   Bar : 
Player o rolled <3, 4>.
|12|13|14|15|16|17|18|19|20|21|22|23|
|--|--|--|--|--|--|--|--|--|--|--|--|
| x|  |  |  |  |  | x| x| x|  | o| x|
|  |  |  |  |  |  | x|  |  |  | o| x|
|  |  |  |  |  |  | x|  |  |  |  | x|
|  |  |  |  |  |  | x|  |  |  |  | x|
|  |  |  |  |  |  |  |  |  |  |  | x|
|  |  |  |  |  |  |  |  |  |  |  | x|


|  | x|  | x|  |  |  |  |  |  |  |  |
|--|--|--|--|--|--|--|--|--|--|--|--|
|11|10| 9| 8| 7| 6| 5| 4| 3| 2| 1| 0|
<Player x>  Off Board :    Bar : 
<Player o>  Off Board

|12|13|14|15|16|17|18|19|20|21|22|23|
|--|--|--|--|--|--|--|--|--|--|--|--|
|  |  |  |  |  |  |  |  |  |  | x| x|
|  |  |  |  |  |  |  |  |  |  | x| x|
|  |  |  |  |  |  |  |  |  |  | x| x|
|  |  |  |  |  |  |  |  |  |  | x| x|
|  |  |  |  |  |  |  |  |  |  |  | x|
|  |  |  |  |  |  |  |  |  |  |  | x|
|  |  |  |  |  |  |  |  |  |  |  | x|


| o|  |  |  |  |  |  |  |  |  |  |  |
|--|--|--|--|--|--|--|--|--|--|--|--|
|11|10| 9| 8| 7| 6| 5| 4| 3| 2| 1| 0|
<Player x>  Off Board : xxxx   Bar : 
<Player o>  Off Board : oooooooooooooo   Bar : 
Player o rolled <6, 5>.
|12|13|14|15|16|17|18|19|20|21|22|23|
|--|--|--|--|--|--|--|--|--|--|--|--|
|  |  |  |  | o|  |  |  |  |  | x| x|
|  |  |  |  |  |  |  |  |  |  | x| x|
|  |  |  |  |  |  |  |  |  |  | x| x|
|  |  |  |  |  |  |  |  |  |  | x| x|
|  |  |  |  |  |  |  |  |  |  |  | x|
|  |  |  |  |  |  |  |  |  |  |  | x|
|  |  |  |  |  |  |  |  |  |  |  | x|


|--|--|--|--|--|--|--|--|--|--|--|--|
|11|10| 9| 8| 7| 6| 5| 4| 3| 2| 1| 0|
<Player x>  

1

In [3]:
# Agents und Modell von https://github.com/fomorians/td-gammon

class RandomAgent(object):

    def __init__(self, player):
        self.player = player
        self.name = 'Random'

    def get_action(self, moves, game=None):
        return random.choice(list(moves)) if moves else None
    
class TDAgent(object):

    def __init__(self, player, model):
        self.player = player
        self.model = model
        self.name = 'TD-Gammon'

    def get_action(self, actions, game):
        """
        Return best action according to self.evaluationFunction,
        with no lookahead.
        """
        v_best = 0
        a_best = None
        for a in actions:

            ateList = game.take_action(a, self.player)
            features = game.extract_features(game.opponent(self.player))
            v = self.model.get_output(features)
            v = 1. - v if self.player == game.players[0] else v
            if v > v_best:
                v_best = v
                a_best = a
            game.undo_action(a, self.player, ateList)

        return a_best


In [5]:
# helper to initialize a weight and bias variable
def weight_bias(shape):
    W = tf.Variable(tf.truncated_normal(shape, stddev=0.1), name='weight')
    b = tf.Variable(tf.constant(0.1, shape=shape[-1:]), name='bias')
    return W, b

# helper to create a dense, fully-connected layer
def dense_layer(x, shape, activation, name):
    with tf.variable_scope(name):
        W, b = weight_bias(shape)
        return activation(tf.matmul(x, W) + b, name='activation')

class Model(object):
    def __init__(self, sess, model_path, summary_path, checkpoint_path, restore=False):
        self.model_path = model_path
        self.summary_path = summary_path
        self.checkpoint_path = checkpoint_path

        # setup our session
        self.sess = sess
        self.global_step = tf.Variable(0, trainable=False, name='global_step')

        # lambda decay
        lamda = tf.maximum(0.7, tf.train.exponential_decay(0.9, self.global_step, \
            30000, 0.96, staircase=True), name='lambda')

        # learning rate decay
        alpha = tf.maximum(0.01, tf.train.exponential_decay(0.1, self.global_step, \
            40000, 0.96, staircase=True), name='alpha')

        tf.summary.scalar('lambda', lamda)
        tf.summary.scalar('alpha', alpha)

        # describe network size
        layer_size_input = 294
        layer_size_hidden = 50
        layer_size_output = 1

        # placeholders for input and target output
        self.x = tf.placeholder('float', [1, layer_size_input], name='x')
        self.V_next = tf.placeholder('float', [1, layer_size_output], name='V_next')

        # build network arch. (just 2 layers with sigmoid activation)
        prev_y = dense_layer(self.x, [layer_size_input, layer_size_hidden], tf.sigmoid, name='layer1')
        self.V = dense_layer(prev_y, [layer_size_hidden, layer_size_output], tf.sigmoid, name='layer2')

        # watch the individual value predictions over time
        tf.summary.scalar('V_next', tf.reduce_sum(self.V_next))
        tf.summary.scalar('V', tf.reduce_sum(self.V))

        # delta = V_next - V
        delta_op = tf.reduce_sum(self.V_next - self.V, name='delta')

        # mean squared error of the difference between the next state and the current state
        loss_op = tf.reduce_mean(tf.square(self.V_next - self.V), name='loss')

        # check if the model predicts the correct state
        accuracy_op = tf.reduce_sum(tf.cast(tf.equal(tf.round(self.V_next), tf.round(self.V)), dtype='float'), name='accuracy')

        # track the number of steps and average loss for the current game
        with tf.variable_scope('game'):
            game_step = tf.Variable(tf.constant(0.0), name='game_step', trainable=False)
            game_step_op = game_step.assign_add(1.0)

            loss_sum = tf.Variable(tf.constant(0.0), name='loss_sum', trainable=False)
            delta_sum = tf.Variable(tf.constant(0.0), name='delta_sum', trainable=False)
            accuracy_sum = tf.Variable(tf.constant(0.0), name='accuracy_sum', trainable=False)

            loss_avg_ema = tf.train.ExponentialMovingAverage(decay=0.999)
            delta_avg_ema = tf.train.ExponentialMovingAverage(decay=0.999)
            accuracy_avg_ema = tf.train.ExponentialMovingAverage(decay=0.999)

            loss_sum_op = loss_sum.assign_add(loss_op)
            delta_sum_op = delta_sum.assign_add(delta_op)
            accuracy_sum_op = accuracy_sum.assign_add(accuracy_op)

            loss_avg_op = loss_sum / tf.maximum(game_step, 1.0)
            delta_avg_op = delta_sum / tf.maximum(game_step, 1.0)
            accuracy_avg_op = accuracy_sum / tf.maximum(game_step, 1.0)

            loss_avg_ema_op = loss_avg_ema.apply([loss_avg_op])
            delta_avg_ema_op = delta_avg_ema.apply([delta_avg_op])
            accuracy_avg_ema_op = accuracy_avg_ema.apply([accuracy_avg_op])

            tf.summary.scalar('game/loss_avg', loss_avg_op)
            tf.summary.scalar('game/delta_avg', delta_avg_op)
            tf.summary.scalar('game/accuracy_avg', accuracy_avg_op)
            tf.summary.scalar('game/loss_avg_ema', loss_avg_ema.average(loss_avg_op))
            tf.summary.scalar('game/delta_avg_ema', delta_avg_ema.average(delta_avg_op))
            tf.summary.scalar('game/accuracy_avg_ema', accuracy_avg_ema.average(accuracy_avg_op))

            # reset per-game monitoring variables
            game_step_reset_op = game_step.assign(0.0)
            loss_sum_reset_op = loss_sum.assign(0.0)
            self.reset_op = tf.group(*[loss_sum_reset_op, game_step_reset_op])

        # increment global step: we keep this as a variable so it's saved with checkpoints
        global_step_op = self.global_step.assign_add(1)

        # get gradients of output V wrt trainable variables (weights and biases)
        tvars = tf.trainable_variables()
        grads = tf.gradients(self.V, tvars)

        # watch the weight and gradient distributions
        for grad, var in zip(grads, tvars):
            tf.summary.histogram(var.name, var)
            tf.summary.histogram(var.name + '/gradients/grad', grad)

        # for each variable, define operations to update the var with delta,
        # taking into account the gradient as part of the eligibility trace
        apply_gradients = []
        with tf.variable_scope('apply_gradients'):
            for grad, var in zip(grads, tvars):
                with tf.variable_scope('trace'):
                    # e-> = lambda * e-> + <grad of output w.r.t weights>
                    trace = tf.Variable(tf.zeros(grad.get_shape()), trainable=False, name='trace')
                    trace_op = trace.assign((lamda * trace) + grad)
                    tf.summary.histogram(var.name + '/traces', trace)

                # grad with trace = alpha * delta * e
                grad_trace = alpha * delta_op * trace_op
                tf.summary.histogram(var.name + '/gradients/trace', grad_trace)

                grad_apply = var.assign_add(grad_trace)
                apply_gradients.append(grad_apply)

        # as part of training we want to update our step and other monitoring variables
        with tf.control_dependencies([
            global_step_op,
            game_step_op,
            loss_sum_op,
            delta_sum_op,
            accuracy_sum_op,
            loss_avg_ema_op,
            delta_avg_ema_op,
            accuracy_avg_ema_op
        ]):
            # define single operation to apply all gradient updates
            self.train_op = tf.group(*apply_gradients, name='train')

        # merge summaries for TensorBoard
        self.summaries_op = tf.summary.merge_all()

        # create a saver for periodic checkpoints
        self.saver = tf.train.Saver(max_to_keep=1)

        # run variable initializers
        self.sess.run(tf.global_variables_initializer())

        # after training a model, we can restore checkpoints here
        if restore:
            self.restore()

    def restore(self):
        latest_checkpoint_path = tf.train.latest_checkpoint(self.checkpoint_path)
        if latest_checkpoint_path:
            print('Restoring checkpoint: {0}'.format(latest_checkpoint_path))
            self.saver.restore(self.sess, latest_checkpoint_path)

    def get_output(self, x):
        return self.sess.run(self.V, feed_dict={ self.x: x })

    def play(self):
        game = Game.new()
        game.play([TDAgent(Game.TOKENS[0], self), HumanAgent(Game.TOKENS[1])], draw=True)

    def test(self, episodes=100, draw=False):
        players = [TDAgent(Game.TOKENS[0], self), RandomAgent(Game.TOKENS[1])]
        winners = [0, 0]
        for episode in range(episodes):
            game = Game.new()

            winner = game.play(players, draw=draw)
            winners[winner] += 1

            winners_total = sum(winners)
            print("[Episode %d] %s (%s) vs %s (%s) %d:%d of %d games (%.2f%%)" % (episode, \
                players[0].name, players[0].player, \
                players[1].name, players[1].player, \
                winners[0], winners[1], winners_total, \
                (winners[0] / winners_total) * 100.0))

    def train(self):
        tf.train.write_graph(self.sess.graph_def, self.model_path, 'td_gammon.pb', as_text=False)
        summary_writer = tf.summary.FileWriter('{0}{1}'.format(self.summary_path, int(time.time()), self.sess.graph_def))

        # the agent plays against itself, making the best move for each player
        players = [TDAgent(Game.TOKENS[0], self), TDAgent(Game.TOKENS[1], self)]

        validation_interval = 1000
        episodes = 5000

        for episode in range(episodes):
            if episode != 0 and episode % validation_interval == 0:
                self.saver.save(self.sess, self.checkpoint_path + 'checkpoint.ckpt', global_step=global_step)
                print("Progress saved!")
                self.test(episodes=100)

            game = Game.new()
            player_num = random.randint(0, 1)

            x = game.extract_features(players[player_num].player)

            game_step = 0
            while not game.is_over():
                game.next_step(players[player_num], player_num)
                player_num = (player_num + 1) % 2

                x_next = game.extract_features(players[player_num].player)
                V_next = self.get_output(x_next)
                self.sess.run(self.train_op, feed_dict={ self.x: x, self.V_next: V_next })

                x = x_next
                game_step += 1

            winner = game.winner()

            _, global_step, summaries, _ = self.sess.run([
                self.train_op,
                self.global_step,
                self.summaries_op,
                self.reset_op
            ], feed_dict={ self.x: x, self.V_next: np.array([[winner]], dtype='float') })
            summary_writer.add_summary(summaries, global_step=global_step)

            print("Game %d/%d (Winner: %s) in %d turns" % (episode, episodes, players[winner].player, game_step))
            

        summary_writer.close()

        self.test(episodes=100)

In [15]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = Model(sess, model_path, summary_path, checkpoint_path)
    model.train()

INFO:tensorflow:Summary name layer1/weight:0 is illegal; using layer1/weight_0 instead.
INFO:tensorflow:Summary name layer1/weight:0/gradients/grad is illegal; using layer1/weight_0/gradients/grad instead.
INFO:tensorflow:Summary name layer1/bias:0 is illegal; using layer1/bias_0 instead.
INFO:tensorflow:Summary name layer1/bias:0/gradients/grad is illegal; using layer1/bias_0/gradients/grad instead.
INFO:tensorflow:Summary name layer2/weight:0 is illegal; using layer2/weight_0 instead.
INFO:tensorflow:Summary name layer2/weight:0/gradients/grad is illegal; using layer2/weight_0/gradients/grad instead.
INFO:tensorflow:Summary name layer2/bias:0 is illegal; using layer2/bias_0 instead.
INFO:tensorflow:Summary name layer2/bias:0/gradients/grad is illegal; using layer2/bias_0/gradients/grad instead.
INFO:tensorflow:Summary name layer1/weight:0/traces is illegal; using layer1/weight_0/traces instead.
INFO:tensorflow:Summary name layer1/weight:0/gradients/trace is illegal; using layer1/weig

Game 173/5000 (Winner: o) in 95 turns
Game 174/5000 (Winner: o) in 83 turns
Game 175/5000 (Winner: o) in 62 turns
Game 176/5000 (Winner: o) in 93 turns
Game 177/5000 (Winner: o) in 87 turns
Game 178/5000 (Winner: x) in 84 turns
Game 179/5000 (Winner: o) in 84 turns
Game 180/5000 (Winner: x) in 71 turns
Game 181/5000 (Winner: x) in 68 turns
Game 182/5000 (Winner: x) in 66 turns
Game 183/5000 (Winner: x) in 71 turns
Game 184/5000 (Winner: x) in 85 turns
Game 185/5000 (Winner: x) in 102 turns
Game 186/5000 (Winner: x) in 74 turns
Game 187/5000 (Winner: x) in 74 turns
Game 188/5000 (Winner: x) in 83 turns
Game 189/5000 (Winner: o) in 75 turns
Game 190/5000 (Winner: x) in 82 turns
Game 191/5000 (Winner: x) in 70 turns
Game 192/5000 (Winner: x) in 82 turns
Game 193/5000 (Winner: x) in 91 turns
Game 194/5000 (Winner: x) in 71 turns
Game 195/5000 (Winner: x) in 88 turns
Game 196/5000 (Winner: o) in 71 turns
Game 197/5000 (Winner: x) in 72 turns
Game 198/5000 (Winner: x) in 61 turns
Game 199/50

Game 389/5000 (Winner: o) in 65 turns
Game 390/5000 (Winner: x) in 57 turns
Game 391/5000 (Winner: x) in 58 turns
Game 392/5000 (Winner: o) in 75 turns
Game 393/5000 (Winner: x) in 61 turns
Game 394/5000 (Winner: o) in 75 turns
Game 395/5000 (Winner: o) in 67 turns
Game 396/5000 (Winner: o) in 56 turns
Game 397/5000 (Winner: o) in 76 turns
Game 398/5000 (Winner: o) in 71 turns
Game 399/5000 (Winner: o) in 65 turns
Game 400/5000 (Winner: x) in 66 turns
Game 401/5000 (Winner: x) in 62 turns
Game 402/5000 (Winner: o) in 83 turns
Game 403/5000 (Winner: o) in 58 turns
Game 404/5000 (Winner: x) in 62 turns
Game 405/5000 (Winner: x) in 73 turns
Game 406/5000 (Winner: o) in 72 turns
Game 407/5000 (Winner: o) in 73 turns
Game 408/5000 (Winner: o) in 71 turns
Game 409/5000 (Winner: o) in 54 turns
Game 410/5000 (Winner: x) in 88 turns
Game 411/5000 (Winner: o) in 71 turns
Game 412/5000 (Winner: o) in 65 turns
Game 413/5000 (Winner: o) in 79 turns
Game 414/5000 (Winner: o) in 70 turns
Game 415/500

Game 605/5000 (Winner: o) in 65 turns
Game 606/5000 (Winner: x) in 70 turns
Game 607/5000 (Winner: x) in 60 turns
Game 608/5000 (Winner: x) in 61 turns
Game 609/5000 (Winner: o) in 83 turns
Game 610/5000 (Winner: o) in 69 turns
Game 611/5000 (Winner: o) in 68 turns
Game 612/5000 (Winner: x) in 57 turns
Game 613/5000 (Winner: x) in 67 turns
Game 614/5000 (Winner: x) in 61 turns
Game 615/5000 (Winner: o) in 64 turns
Game 616/5000 (Winner: x) in 63 turns
Game 617/5000 (Winner: x) in 64 turns
Game 618/5000 (Winner: x) in 68 turns
Game 619/5000 (Winner: o) in 74 turns
Game 620/5000 (Winner: x) in 77 turns
Game 621/5000 (Winner: o) in 54 turns
Game 622/5000 (Winner: x) in 58 turns
Game 623/5000 (Winner: x) in 66 turns
Game 624/5000 (Winner: o) in 55 turns
Game 625/5000 (Winner: o) in 52 turns
Game 626/5000 (Winner: o) in 50 turns
Game 627/5000 (Winner: o) in 62 turns
Game 628/5000 (Winner: o) in 67 turns
Game 629/5000 (Winner: x) in 54 turns
Game 630/5000 (Winner: x) in 63 turns
Game 631/500

Game 821/5000 (Winner: x) in 63 turns
Game 822/5000 (Winner: x) in 59 turns
Game 823/5000 (Winner: o) in 62 turns
Game 824/5000 (Winner: o) in 62 turns
Game 825/5000 (Winner: x) in 72 turns
Game 826/5000 (Winner: o) in 81 turns
Game 827/5000 (Winner: o) in 63 turns
Game 828/5000 (Winner: x) in 72 turns
Game 829/5000 (Winner: x) in 63 turns
Game 830/5000 (Winner: x) in 61 turns
Game 831/5000 (Winner: x) in 61 turns
Game 832/5000 (Winner: o) in 53 turns
Game 833/5000 (Winner: o) in 61 turns
Game 834/5000 (Winner: o) in 47 turns
Game 835/5000 (Winner: o) in 76 turns
Game 836/5000 (Winner: o) in 61 turns
Game 837/5000 (Winner: o) in 77 turns
Game 838/5000 (Winner: o) in 53 turns
Game 839/5000 (Winner: o) in 70 turns
Game 840/5000 (Winner: o) in 51 turns
Game 841/5000 (Winner: o) in 59 turns
Game 842/5000 (Winner: x) in 57 turns
Game 843/5000 (Winner: o) in 71 turns
Game 844/5000 (Winner: o) in 63 turns
Game 845/5000 (Winner: o) in 76 turns
Game 846/5000 (Winner: o) in 53 turns
Game 847/500

...

Game 4327/5000 (Winner: x) in 71 turns
Game 4328/5000 (Winner: o) in 121 turns
Game 4329/5000 (Winner: o) in 137 turns
Game 4330/5000 (Winner: x) in 76 turns
Game 4331/5000 (Winner: o) in 97 turns
Game 4332/5000 (Winner: x) in 100 turns
Game 4333/5000 (Winner: o) in 82 turns
Game 4334/5000 (Winner: o) in 103 turns
Game 4335/5000 (Winner: x) in 83 turns
Game 4336/5000 (Winner: o) in 95 turns
Game 4337/5000 (Winner: x) in 92 turns
Game 4338/5000 (Winner: o) in 111 turns
Game 4339/5000 (Winner: x) in 95 turns
Game 4340/5000 (Winner: o) in 110 turns
Game 4341/5000 (Winner: o) in 148 turns
Game 4342/5000 (Winner: o) in 93 turns
Game 4343/5000 (Winner: o) in 91 turns
Game 4344/5000 (Winner: x) in 80 turns
Game 4345/5000 (Winner: x) in 91 turns
Game 4346/5000 (Winner: o) in 91 turns
Game 4347/5000 (Winner: x) in 100 turns
Game 4348/5000 (Winner: o) in 144 turns
Game 4349/5000 (Winner: o) in 107 turns
Game 4350/5000 (Winner: o) in 92 turns
Game 4351/5000 (Winner: x) in 112 turns
Game 4352/5000

Game 4535/5000 (Winner: x) in 79 turns
Game 4536/5000 (Winner: o) in 161 turns
Game 4537/5000 (Winner: o) in 86 turns
Game 4538/5000 (Winner: x) in 89 turns
Game 4539/5000 (Winner: o) in 108 turns
Game 4540/5000 (Winner: o) in 77 turns
Game 4541/5000 (Winner: x) in 90 turns
Game 4542/5000 (Winner: o) in 117 turns
Game 4543/5000 (Winner: o) in 98 turns
Game 4544/5000 (Winner: o) in 135 turns
Game 4545/5000 (Winner: o) in 169 turns
Game 4546/5000 (Winner: o) in 88 turns
Game 4547/5000 (Winner: o) in 76 turns
Game 4548/5000 (Winner: o) in 99 turns
Game 4549/5000 (Winner: o) in 103 turns
Game 4550/5000 (Winner: o) in 83 turns
Game 4551/5000 (Winner: x) in 97 turns
Game 4552/5000 (Winner: o) in 103 turns
Game 4553/5000 (Winner: o) in 121 turns
Game 4554/5000 (Winner: o) in 121 turns
Game 4555/5000 (Winner: o) in 132 turns
Game 4556/5000 (Winner: o) in 49 turns
Game 4557/5000 (Winner: o) in 116 turns
Game 4558/5000 (Winner: o) in 103 turns
Game 4559/5000 (Winner: x) in 59 turns
Game 4560/500

Game 4744/5000 (Winner: x) in 95 turns
Game 4745/5000 (Winner: o) in 123 turns
Game 4746/5000 (Winner: o) in 111 turns
Game 4747/5000 (Winner: x) in 107 turns
Game 4748/5000 (Winner: x) in 101 turns
Game 4749/5000 (Winner: x) in 107 turns
Game 4750/5000 (Winner: x) in 104 turns
Game 4751/5000 (Winner: o) in 103 turns
Game 4752/5000 (Winner: o) in 86 turns
Game 4753/5000 (Winner: x) in 114 turns
Game 4754/5000 (Winner: x) in 88 turns
Game 4755/5000 (Winner: o) in 99 turns
Game 4756/5000 (Winner: o) in 92 turns
Game 4757/5000 (Winner: x) in 100 turns
Game 4758/5000 (Winner: o) in 110 turns
Game 4759/5000 (Winner: x) in 96 turns
Game 4760/5000 (Winner: o) in 72 turns
Game 4761/5000 (Winner: o) in 66 turns
Game 4762/5000 (Winner: o) in 82 turns
Game 4763/5000 (Winner: o) in 101 turns
Game 4764/5000 (Winner: o) in 63 turns
Game 4765/5000 (Winner: o) in 94 turns
Game 4766/5000 (Winner: o) in 95 turns
Game 4767/5000 (Winner: x) in 99 turns
Game 4768/5000 (Winner: o) in 111 turns
Game 4769/500

Game 4953/5000 (Winner: o) in 106 turns
Game 4954/5000 (Winner: o) in 72 turns
Game 4955/5000 (Winner: x) in 71 turns
Game 4956/5000 (Winner: o) in 72 turns
Game 4957/5000 (Winner: o) in 80 turns
Game 4958/5000 (Winner: o) in 109 turns
Game 4959/5000 (Winner: o) in 95 turns
Game 4960/5000 (Winner: o) in 90 turns
Game 4961/5000 (Winner: o) in 99 turns
Game 4962/5000 (Winner: o) in 91 turns
Game 4963/5000 (Winner: o) in 105 turns
Game 4964/5000 (Winner: x) in 120 turns
Game 4965/5000 (Winner: x) in 98 turns
Game 4966/5000 (Winner: x) in 98 turns
Game 4967/5000 (Winner: x) in 94 turns
Game 4968/5000 (Winner: o) in 149 turns
Game 4969/5000 (Winner: o) in 83 turns
Game 4970/5000 (Winner: x) in 104 turns
Game 4971/5000 (Winner: o) in 74 turns
Game 4972/5000 (Winner: x) in 91 turns
Game 4973/5000 (Winner: o) in 67 turns
Game 4974/5000 (Winner: o) in 98 turns
Game 4975/5000 (Winner: o) in 117 turns
Game 4976/5000 (Winner: o) in 97 turns
Game 4977/5000 (Winner: x) in 90 turns
Game 4978/5000 (Wi

[Episode 95] TD-Gammon (x) vs Random (o) 94:2 of 96 games (97.92%)
[Episode 96] TD-Gammon (x) vs Random (o) 95:2 of 97 games (97.94%)
[Episode 97] TD-Gammon (x) vs Random (o) 96:2 of 98 games (97.96%)
[Episode 98] TD-Gammon (x) vs Random (o) 97:2 of 99 games (97.98%)
[Episode 99] TD-Gammon (x) vs Random (o) 98:2 of 100 games (98.00%)


In [44]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = Model(sess, model_path, summary_path, checkpoint_path, restore=True)
    model.test(episodes=1000)

INFO:tensorflow:Summary name layer1/weight:0 is illegal; using layer1/weight_0 instead.
INFO:tensorflow:Summary name layer1/weight:0/gradients/grad is illegal; using layer1/weight_0/gradients/grad instead.
INFO:tensorflow:Summary name layer1/bias:0 is illegal; using layer1/bias_0 instead.
INFO:tensorflow:Summary name layer1/bias:0/gradients/grad is illegal; using layer1/bias_0/gradients/grad instead.
INFO:tensorflow:Summary name layer2/weight:0 is illegal; using layer2/weight_0 instead.
INFO:tensorflow:Summary name layer2/weight:0/gradients/grad is illegal; using layer2/weight_0/gradients/grad instead.
INFO:tensorflow:Summary name layer2/bias:0 is illegal; using layer2/bias_0 instead.
INFO:tensorflow:Summary name layer2/bias:0/gradients/grad is illegal; using layer2/bias_0/gradients/grad instead.
INFO:tensorflow:Summary name layer1/weight:0/traces is illegal; using layer1/weight_0/traces instead.
INFO:tensorflow:Summary name layer1/weight:0/gradients/trace is illegal; using layer1/weig

[Episode 95] TD-Gammon (x) vs Random (o) 96:0 of 96 games (100.00%)
[Episode 96] TD-Gammon (x) vs Random (o) 97:0 of 97 games (100.00%)
[Episode 97] TD-Gammon (x) vs Random (o) 98:0 of 98 games (100.00%)
[Episode 98] TD-Gammon (x) vs Random (o) 99:0 of 99 games (100.00%)
[Episode 99] TD-Gammon (x) vs Random (o) 100:0 of 100 games (100.00%)
[Episode 100] TD-Gammon (x) vs Random (o) 101:0 of 101 games (100.00%)
[Episode 101] TD-Gammon (x) vs Random (o) 102:0 of 102 games (100.00%)
[Episode 102] TD-Gammon (x) vs Random (o) 103:0 of 103 games (100.00%)
[Episode 103] TD-Gammon (x) vs Random (o) 104:0 of 104 games (100.00%)
[Episode 104] TD-Gammon (x) vs Random (o) 105:0 of 105 games (100.00%)
[Episode 105] TD-Gammon (x) vs Random (o) 106:0 of 106 games (100.00%)
[Episode 106] TD-Gammon (x) vs Random (o) 107:0 of 107 games (100.00%)
[Episode 107] TD-Gammon (x) vs Random (o) 108:0 of 108 games (100.00%)
[Episode 108] TD-Gammon (x) vs Random (o) 109:0 of 109 games (100.00%)
[Episode 109] TD-Ga

[Episode 211] TD-Gammon (x) vs Random (o) 212:0 of 212 games (100.00%)
[Episode 212] TD-Gammon (x) vs Random (o) 213:0 of 213 games (100.00%)
[Episode 213] TD-Gammon (x) vs Random (o) 214:0 of 214 games (100.00%)
[Episode 214] TD-Gammon (x) vs Random (o) 215:0 of 215 games (100.00%)
[Episode 215] TD-Gammon (x) vs Random (o) 216:0 of 216 games (100.00%)
[Episode 216] TD-Gammon (x) vs Random (o) 217:0 of 217 games (100.00%)
[Episode 217] TD-Gammon (x) vs Random (o) 218:0 of 218 games (100.00%)
[Episode 218] TD-Gammon (x) vs Random (o) 219:0 of 219 games (100.00%)
[Episode 219] TD-Gammon (x) vs Random (o) 220:0 of 220 games (100.00%)
[Episode 220] TD-Gammon (x) vs Random (o) 221:0 of 221 games (100.00%)
[Episode 221] TD-Gammon (x) vs Random (o) 222:0 of 222 games (100.00%)
[Episode 222] TD-Gammon (x) vs Random (o) 223:0 of 223 games (100.00%)
[Episode 223] TD-Gammon (x) vs Random (o) 224:0 of 224 games (100.00%)
[Episode 224] TD-Gammon (x) vs Random (o) 225:0 of 225 games (100.00%)
[Episo

[Episode 327] TD-Gammon (x) vs Random (o) 327:1 of 328 games (99.70%)
[Episode 328] TD-Gammon (x) vs Random (o) 328:1 of 329 games (99.70%)
[Episode 329] TD-Gammon (x) vs Random (o) 329:1 of 330 games (99.70%)
[Episode 330] TD-Gammon (x) vs Random (o) 330:1 of 331 games (99.70%)
[Episode 331] TD-Gammon (x) vs Random (o) 331:1 of 332 games (99.70%)
[Episode 332] TD-Gammon (x) vs Random (o) 332:1 of 333 games (99.70%)
[Episode 333] TD-Gammon (x) vs Random (o) 333:1 of 334 games (99.70%)
[Episode 334] TD-Gammon (x) vs Random (o) 334:1 of 335 games (99.70%)
[Episode 335] TD-Gammon (x) vs Random (o) 335:1 of 336 games (99.70%)
[Episode 336] TD-Gammon (x) vs Random (o) 336:1 of 337 games (99.70%)
[Episode 337] TD-Gammon (x) vs Random (o) 337:1 of 338 games (99.70%)
[Episode 338] TD-Gammon (x) vs Random (o) 337:2 of 339 games (99.41%)
[Episode 339] TD-Gammon (x) vs Random (o) 338:2 of 340 games (99.41%)
[Episode 340] TD-Gammon (x) vs Random (o) 339:2 of 341 games (99.41%)
[Episode 341] TD-Gam

[Episode 445] TD-Gammon (x) vs Random (o) 443:3 of 446 games (99.33%)
[Episode 446] TD-Gammon (x) vs Random (o) 444:3 of 447 games (99.33%)
[Episode 447] TD-Gammon (x) vs Random (o) 445:3 of 448 games (99.33%)
[Episode 448] TD-Gammon (x) vs Random (o) 446:3 of 449 games (99.33%)
[Episode 449] TD-Gammon (x) vs Random (o) 447:3 of 450 games (99.33%)
[Episode 450] TD-Gammon (x) vs Random (o) 448:3 of 451 games (99.33%)
[Episode 451] TD-Gammon (x) vs Random (o) 449:3 of 452 games (99.34%)
[Episode 452] TD-Gammon (x) vs Random (o) 450:3 of 453 games (99.34%)
[Episode 453] TD-Gammon (x) vs Random (o) 451:3 of 454 games (99.34%)
[Episode 454] TD-Gammon (x) vs Random (o) 452:3 of 455 games (99.34%)
[Episode 455] TD-Gammon (x) vs Random (o) 453:3 of 456 games (99.34%)
[Episode 456] TD-Gammon (x) vs Random (o) 454:3 of 457 games (99.34%)
[Episode 457] TD-Gammon (x) vs Random (o) 455:3 of 458 games (99.34%)
[Episode 458] TD-Gammon (x) vs Random (o) 456:3 of 459 games (99.35%)
[Episode 459] TD-Gam

[Episode 563] TD-Gammon (x) vs Random (o) 561:3 of 564 games (99.47%)
[Episode 564] TD-Gammon (x) vs Random (o) 562:3 of 565 games (99.47%)
[Episode 565] TD-Gammon (x) vs Random (o) 563:3 of 566 games (99.47%)
[Episode 566] TD-Gammon (x) vs Random (o) 564:3 of 567 games (99.47%)
[Episode 567] TD-Gammon (x) vs Random (o) 565:3 of 568 games (99.47%)
[Episode 568] TD-Gammon (x) vs Random (o) 566:3 of 569 games (99.47%)
[Episode 569] TD-Gammon (x) vs Random (o) 567:3 of 570 games (99.47%)
[Episode 570] TD-Gammon (x) vs Random (o) 568:3 of 571 games (99.47%)
[Episode 571] TD-Gammon (x) vs Random (o) 569:3 of 572 games (99.48%)
[Episode 572] TD-Gammon (x) vs Random (o) 570:3 of 573 games (99.48%)
[Episode 573] TD-Gammon (x) vs Random (o) 571:3 of 574 games (99.48%)
[Episode 574] TD-Gammon (x) vs Random (o) 572:3 of 575 games (99.48%)
[Episode 575] TD-Gammon (x) vs Random (o) 573:3 of 576 games (99.48%)
[Episode 576] TD-Gammon (x) vs Random (o) 574:3 of 577 games (99.48%)
[Episode 577] TD-Gam

[Episode 682] TD-Gammon (x) vs Random (o) 680:3 of 683 games (99.56%)
[Episode 683] TD-Gammon (x) vs Random (o) 681:3 of 684 games (99.56%)
[Episode 684] TD-Gammon (x) vs Random (o) 682:3 of 685 games (99.56%)
[Episode 685] TD-Gammon (x) vs Random (o) 683:3 of 686 games (99.56%)
[Episode 686] TD-Gammon (x) vs Random (o) 684:3 of 687 games (99.56%)
[Episode 687] TD-Gammon (x) vs Random (o) 685:3 of 688 games (99.56%)
[Episode 688] TD-Gammon (x) vs Random (o) 686:3 of 689 games (99.56%)
[Episode 689] TD-Gammon (x) vs Random (o) 687:3 of 690 games (99.57%)
[Episode 690] TD-Gammon (x) vs Random (o) 688:3 of 691 games (99.57%)
[Episode 691] TD-Gammon (x) vs Random (o) 689:3 of 692 games (99.57%)
[Episode 692] TD-Gammon (x) vs Random (o) 690:3 of 693 games (99.57%)
[Episode 693] TD-Gammon (x) vs Random (o) 691:3 of 694 games (99.57%)
[Episode 694] TD-Gammon (x) vs Random (o) 692:3 of 695 games (99.57%)
[Episode 695] TD-Gammon (x) vs Random (o) 693:3 of 696 games (99.57%)
[Episode 696] TD-Gam

[Episode 801] TD-Gammon (x) vs Random (o) 799:3 of 802 games (99.63%)
[Episode 802] TD-Gammon (x) vs Random (o) 800:3 of 803 games (99.63%)
[Episode 803] TD-Gammon (x) vs Random (o) 801:3 of 804 games (99.63%)
[Episode 804] TD-Gammon (x) vs Random (o) 802:3 of 805 games (99.63%)
[Episode 805] TD-Gammon (x) vs Random (o) 803:3 of 806 games (99.63%)
[Episode 806] TD-Gammon (x) vs Random (o) 804:3 of 807 games (99.63%)
[Episode 807] TD-Gammon (x) vs Random (o) 805:3 of 808 games (99.63%)
[Episode 808] TD-Gammon (x) vs Random (o) 806:3 of 809 games (99.63%)
[Episode 809] TD-Gammon (x) vs Random (o) 807:3 of 810 games (99.63%)
[Episode 810] TD-Gammon (x) vs Random (o) 808:3 of 811 games (99.63%)
[Episode 811] TD-Gammon (x) vs Random (o) 809:3 of 812 games (99.63%)
[Episode 812] TD-Gammon (x) vs Random (o) 810:3 of 813 games (99.63%)
[Episode 813] TD-Gammon (x) vs Random (o) 811:3 of 814 games (99.63%)
[Episode 814] TD-Gammon (x) vs Random (o) 812:3 of 815 games (99.63%)
[Episode 815] TD-Gam

[Episode 919] TD-Gammon (x) vs Random (o) 917:3 of 920 games (99.67%)
[Episode 920] TD-Gammon (x) vs Random (o) 918:3 of 921 games (99.67%)
[Episode 921] TD-Gammon (x) vs Random (o) 919:3 of 922 games (99.67%)
[Episode 922] TD-Gammon (x) vs Random (o) 920:3 of 923 games (99.67%)
[Episode 923] TD-Gammon (x) vs Random (o) 921:3 of 924 games (99.68%)
[Episode 924] TD-Gammon (x) vs Random (o) 922:3 of 925 games (99.68%)
[Episode 925] TD-Gammon (x) vs Random (o) 923:3 of 926 games (99.68%)
[Episode 926] TD-Gammon (x) vs Random (o) 924:3 of 927 games (99.68%)
[Episode 927] TD-Gammon (x) vs Random (o) 925:3 of 928 games (99.68%)
[Episode 928] TD-Gammon (x) vs Random (o) 926:3 of 929 games (99.68%)
[Episode 929] TD-Gammon (x) vs Random (o) 927:3 of 930 games (99.68%)
[Episode 930] TD-Gammon (x) vs Random (o) 928:3 of 931 games (99.68%)
[Episode 931] TD-Gammon (x) vs Random (o) 929:3 of 932 games (99.68%)
[Episode 932] TD-Gammon (x) vs Random (o) 930:3 of 933 games (99.68%)
[Episode 933] TD-Gam

In [31]:
"""
Eigener TDGammon Versuch

Der Agent sucht nur seinen eigenen Halbzug ab (1-ply)

Das Modell folgt dem Paper "Temporal Difference Learning and TD-Gammon" von Gerald Tesauro

"""

class TDAgent2(object):

    def __init__(self, player, model, new=True):
        self.player = player
        self.model = model
        self.name = 'TD-Gammon0.0'
        self.newGame = new

    def get_action(self, actions, game):
        v_best = 0
        a_best = None
        #Durchsucht alle gültigen, mögliche Züge nach dem Zug ab,
        #der für das Modell die höchste Siegwahrscheinlichkeit hat
        for a in actions:
            if self.newGame:
                old_state = game.execute_moves(a, self.player)
            else:
                ateList = game.take_action(a, self.player)
            features = game.extractFeatures(self.player)
            v = self.model.get_output(features)
            v = 1. - v if self.player == game.players[0] else v
            if v > v_best:
                v_best = v
                a_best = a
            if self.newGame:
                game.reset_to_state(old_state)
            else:
                game.undo_action(a, self.player, ateList)
        return a_best

class OwnModel(object):
    def __init__(self, sess, checkpoint_path, restore=False):
        self.checkpoint_path = checkpoint_path
        
        #session
        self.sess = sess
        self.global_step = tf.Variable(0, trainable=False, name='global_step')

        #lambda, leider ohne b da dies ein Schlüsselwort ist
        lamda = 0.7

        #learning rate 
        alpha = 0.1

        #Für leichteres abändern der Netzwerkarchitektur
        layer_size_input = 198
        layer_size_hidden = 40
        layer_size_output = 1

        # placeholders for input and target output
        self.x = tf.placeholder('float', [1, layer_size_input], name='x')
        self.V_next = tf.placeholder('float', [1, layer_size_output], name='V_next')

        #Zwei fully-connected, dense Layer: Ein Input-Layer (198 Units) und dann ein Hidden-Layer (40 Units)
        #Beide werden mit der sigmoid Funktion aktiviert
        prev_y = tf.layers.dense(self.x, layer_size_hidden, activation= tf.sigmoid)
        self.V = tf.layers.dense(prev_y, layer_size_output, activation= tf.sigmoid)
        
        # delta = V_next - V
        delta_op = tf.reduce_sum(self.V_next - self.V, name='delta')

        #Global_step wird bei jeden aufruf von sess.run um 1 erhöht
        global_step_op = self.global_step.assign_add(1)

        # get gradients of output V wrt trainable variables (weights and biases)
        tvars = tf.trainable_variables()
        grads = tf.gradients(self.V, tvars)

        #Alle Variablen werden angepasst und mittels eligebility traces wird er Wert
        #der einzelnen Gewichte angepasst um gute bzw. schlechte Entscheidungen zu reflektieren
        apply_gradients = []

        for grad, var in zip(grads, tvars):
            # e-> = lambda * e-> + <grad of output w.r.t weights>
            trace = tf.Variable(tf.zeros(grad.get_shape()), trainable=False, name='trace')
            trace_op = trace.assign((lamda * trace) + grad)

            # grad with trace = alpha * delta * e
            grad_trace = alpha * delta_op * trace_op

            grad_apply = var.assign_add(grad_trace)
            apply_gradients.append(grad_apply)
        
        #Den global_step mitzählen lassen
        apply_gradients.append(global_step_op)

        #Alle Gradientenoperationen in einer train Operation zusammenfassen
        self.train_op = tf.group(*apply_gradients, name='train')
        
        #Der Saver behält immer nur den neusten Checkpoint
        self.saver = tf.train.Saver(max_to_keep=1)

        #Variablen initialisieren 
        self.sess.run(tf.global_variables_initializer())

        #Modell wiederherstellen falls gewünscht
        if restore:
            self.restore()

    #Lädt den neusten (und auch einzigen) checkpoint 
    def restore(self):
        latest_checkpoint_path = tf.train.latest_checkpoint(self.checkpoint_path)
        if latest_checkpoint_path:
            print('Restoring checkpoint: {0}'.format(latest_checkpoint_path))
            self.saver.restore(self.sess, latest_checkpoint_path)

    #Erzeugt einen Outpt für die gegebenen features x
    def get_output(self, x):
        return self.sess.run(self.V, feed_dict={ self.x: x })

    #Testet das Modell gegen den angegebenen enemyAgent auf der neuen Backgammon Engine
    def test(self, enemyAgent=RandomAgent('white'), episodes=100, debug=False):
        players = [TDAgent2('black', self), enemyAgent]
        
        winners = {'black':0, 'white':0}
        for episode in range(episodes):
            game = OwnGame()

            winner = game.play(players, debug=debug)
            winners[winner] += 1

            winners_total = sum(winners.values())
            print("[Episode %d] %s (%s) vs %s (%s) %d:%d of %d games (%.2f%%)" % (episode, \
                players[0].name, players[0].player, \
                players[1].name, players[1].player, \
                winners['black'], winners['white'], winners_total, \
                (winners['black'] / winners_total) * 100.0))
            
    #Battle gegen ein Modell auf der alten Spielimplementierung
    def battle(self, enemyAgent, episodes=100, draw=False):
        players = [TDAgent2(Game.TOKENS[0], self, new=False), enemyAgent]
        winners = [0, 0]
        for episode in range(episodes):
            game = Game.new()

            winner = game.play(players, draw=draw)
            winners[winner] += 1

            winners_total = sum(winners)
            print("[Episode %d] %s (%s) vs %s (%s) %d:%d of %d games (%.2f%%)" % (episode, \
                players[0].name, players[0].player, \
                players[1].name, players[1].player, \
                winners[0], winners[1], winners_total, \
                (winners[0] / winners_total) * 100.0))

    def train(self, episodes, validation_interval, test_episodes=100):
        #Selbsttraining, Modell vs Modell
        players = [TDAgent2('black', self), TDAgent2('white', self)]

        for episode in range(episodes):
            #Immer wieder zwischendurch testen und den Fortschritt speichern
            if episode != 0 and episode % validation_interval == 0:
                self.saver.save(self.sess, self.checkpoint_path + 'checkpoint ' + str(global_step) + '.ckpt', global_step=global_step)
                print("Progress saved!")
                self.test(episodes=test_episodes)

            #Spiel initialisieren
            game = OwnGame()
            player_num = random.randint(0, 1)
            
            x = game.extractFeatures(players[player_num].player)

            #Spiel spielen bis es einen Sieger gibt
            game_step = 0
            while not game.get_winner():
                game.next_step(players[player_num], player_num)
                player_num = (player_num + 1) % 2

                #Das Modell mit jeden Schritt im Spiel trainieren
                x_next = game.extractFeatures(players[player_num].player)
                V_next = self.get_output(x_next)
                
                self.sess.run(self.train_op, feed_dict={ self.x: x, self.V_next: V_next })

                x = x_next
                game_step += 1

            #Gewinner ermitteln
            winner = 0 if game.get_winner() == game.players[0] else 1

            #Zu guter letzt reinforcement learning: Dem Modell noch eine "Belohnung" geben, wenn es gewonnen hat
            _, global_step = self.sess.run([
                self.train_op,
                self.global_step,
            ], feed_dict={ self.x: x, self.V_next: np.array([[winner]], dtype='float') })

            #Konsolenausgabe hübsch aufbereiten
            print("Game %d/%d (Winner: %s) in %d turns" % (episode, episodes, players[winner].player, game_step))
        #Am Ende noch mal speichern und 100 testen!
        self.saver.save(self.sess, self.checkpoint_path + 'checkpoint ' + str(global_step) + '.ckpt', global_step=global_step) 
        self.test(episodes=test_episodes)

In [32]:
#Diese Zelle ausführen um das Modell weiter trainieren zu lassen
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = OwnModel(sess, own_checkpoint_path, restore=True)
    #model.train(1000,1000)
    model.test()

Restoring checkpoint: own_checkpoints/checkpoint 1146457.ckpt-1146457
INFO:tensorflow:Restoring parameters from own_checkpoints/checkpoint 1146457.ckpt-1146457
[Episode 0] TD-Gammon0.0 (black) vs Random (white) 1:0 of 1 games (100.00%)
[Episode 1] TD-Gammon0.0 (black) vs Random (white) 2:0 of 2 games (100.00%)
[Episode 2] TD-Gammon0.0 (black) vs Random (white) 3:0 of 3 games (100.00%)
[Episode 3] TD-Gammon0.0 (black) vs Random (white) 4:0 of 4 games (100.00%)
[Episode 4] TD-Gammon0.0 (black) vs Random (white) 5:0 of 5 games (100.00%)
[Episode 5] TD-Gammon0.0 (black) vs Random (white) 6:0 of 6 games (100.00%)
[Episode 6] TD-Gammon0.0 (black) vs Random (white) 7:0 of 7 games (100.00%)
[Episode 7] TD-Gammon0.0 (black) vs Random (white) 8:0 of 8 games (100.00%)
[Episode 8] TD-Gammon0.0 (black) vs Random (white) 9:0 of 9 games (100.00%)
[Episode 9] TD-Gammon0.0 (black) vs Random (white) 10:0 of 10 games (100.00%)
[Episode 10] TD-Gammon0.0 (black) vs Random (white) 11:0 of 11 games (100.00%)

Das Modell erreicht nach 6000 Spielen 100% Erfolgsquote gegen einen zufälligen Spieler, was daraufhin deutet, dass es eine Strategie gelernt hat!

In [189]:
import cProfile

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = OwnModel(sess, own_checkpoint_path, restore=True)
    cProfile.run('model.train(1000,1000)')

Restoring checkpoint: own_checkpoints/checkpoint 600907.ckpt-600907
INFO:tensorflow:Restoring parameters from own_checkpoints/checkpoint 600907.ckpt-600907
Game 0/1000 (Winner: white) in 103 turns
Game 1/1000 (Winner: white) in 106 turns
Game 2/1000 (Winner: black) in 135 turns
Game 3/1000 (Winner: black) in 101 turns
Game 4/1000 (Winner: white) in 99 turns
Game 5/1000 (Winner: white) in 228 turns
Game 6/1000 (Winner: black) in 91 turns
Game 7/1000 (Winner: black) in 94 turns
Game 8/1000 (Winner: white) in 98 turns
Game 9/1000 (Winner: black) in 121 turns
Game 10/1000 (Winner: black) in 56 turns
Game 11/1000 (Winner: white) in 144 turns
Game 12/1000 (Winner: white) in 117 turns
Game 13/1000 (Winner: white) in 116 turns
Game 14/1000 (Winner: black) in 70 turns
Game 15/1000 (Winner: white) in 61 turns
Game 16/1000 (Winner: black) in 113 turns
Game 17/1000 (Winner: white) in 99 turns
Game 18/1000 (Winner: black) in 108 turns
Game 19/1000 (Winner: white) in 83 turns
Game 20/1000 (Winner: w

Game 192/1000 (Winner: white) in 156 turns
Game 193/1000 (Winner: white) in 76 turns
Game 194/1000 (Winner: black) in 90 turns
Game 195/1000 (Winner: black) in 105 turns
Game 196/1000 (Winner: black) in 123 turns
Game 197/1000 (Winner: black) in 165 turns
Game 198/1000 (Winner: white) in 106 turns
Game 199/1000 (Winner: black) in 115 turns
Game 200/1000 (Winner: black) in 90 turns
Game 201/1000 (Winner: black) in 167 turns
Game 202/1000 (Winner: white) in 57 turns
Game 203/1000 (Winner: black) in 102 turns
Game 204/1000 (Winner: white) in 73 turns
Game 205/1000 (Winner: black) in 111 turns
Game 206/1000 (Winner: white) in 58 turns
Game 207/1000 (Winner: black) in 80 turns
Game 208/1000 (Winner: black) in 83 turns
Game 209/1000 (Winner: white) in 112 turns
Game 210/1000 (Winner: black) in 136 turns
Game 211/1000 (Winner: black) in 154 turns
Game 212/1000 (Winner: black) in 139 turns
Game 213/1000 (Winner: white) in 53 turns
Game 214/1000 (Winner: black) in 78 turns
Game 215/1000 (Winner

Game 385/1000 (Winner: white) in 63 turns
Game 386/1000 (Winner: white) in 73 turns
Game 387/1000 (Winner: black) in 178 turns
Game 388/1000 (Winner: white) in 207 turns
Game 389/1000 (Winner: black) in 160 turns
Game 390/1000 (Winner: white) in 141 turns
Game 391/1000 (Winner: black) in 231 turns
Game 392/1000 (Winner: black) in 115 turns
Game 393/1000 (Winner: black) in 136 turns
Game 394/1000 (Winner: black) in 107 turns
Game 395/1000 (Winner: black) in 86 turns
Game 396/1000 (Winner: black) in 137 turns
Game 397/1000 (Winner: white) in 136 turns
Game 398/1000 (Winner: black) in 92 turns
Game 399/1000 (Winner: white) in 169 turns
Game 400/1000 (Winner: black) in 145 turns
Game 401/1000 (Winner: white) in 137 turns
Game 402/1000 (Winner: white) in 176 turns
Game 403/1000 (Winner: black) in 62 turns
Game 404/1000 (Winner: white) in 57 turns
Game 405/1000 (Winner: white) in 88 turns
Game 406/1000 (Winner: black) in 77 turns
Game 407/1000 (Winner: black) in 277 turns
Game 408/1000 (Winn

Game 578/1000 (Winner: white) in 106 turns
Game 579/1000 (Winner: black) in 128 turns
Game 580/1000 (Winner: black) in 66 turns
Game 581/1000 (Winner: black) in 73 turns
Game 582/1000 (Winner: white) in 142 turns
Game 583/1000 (Winner: black) in 135 turns
Game 584/1000 (Winner: white) in 65 turns
Game 585/1000 (Winner: black) in 110 turns
Game 586/1000 (Winner: white) in 84 turns
Game 587/1000 (Winner: white) in 114 turns
Game 588/1000 (Winner: white) in 201 turns
Game 589/1000 (Winner: white) in 154 turns
Game 590/1000 (Winner: black) in 164 turns
Game 591/1000 (Winner: black) in 156 turns
Game 592/1000 (Winner: white) in 161 turns
Game 593/1000 (Winner: black) in 89 turns
Game 594/1000 (Winner: white) in 121 turns
Game 595/1000 (Winner: white) in 73 turns
Game 596/1000 (Winner: black) in 224 turns
Game 597/1000 (Winner: black) in 157 turns
Game 598/1000 (Winner: black) in 82 turns
Game 599/1000 (Winner: white) in 114 turns
Game 600/1000 (Winner: white) in 176 turns
Game 601/1000 (Win

Game 771/1000 (Winner: black) in 82 turns
Game 772/1000 (Winner: black) in 85 turns
Game 773/1000 (Winner: white) in 82 turns
Game 774/1000 (Winner: white) in 141 turns
Game 775/1000 (Winner: black) in 125 turns
Game 776/1000 (Winner: white) in 115 turns
Game 777/1000 (Winner: white) in 164 turns
Game 778/1000 (Winner: white) in 55 turns
Game 779/1000 (Winner: black) in 77 turns
Game 780/1000 (Winner: white) in 81 turns
Game 781/1000 (Winner: black) in 124 turns
Game 782/1000 (Winner: black) in 170 turns
Game 783/1000 (Winner: white) in 169 turns
Game 784/1000 (Winner: black) in 80 turns
Game 785/1000 (Winner: black) in 113 turns
Game 786/1000 (Winner: white) in 103 turns
Game 787/1000 (Winner: white) in 207 turns
Game 788/1000 (Winner: white) in 66 turns
Game 789/1000 (Winner: black) in 224 turns
Game 790/1000 (Winner: white) in 80 turns
Game 791/1000 (Winner: black) in 79 turns
Game 792/1000 (Winner: black) in 156 turns
Game 793/1000 (Winner: black) in 57 turns
Game 794/1000 (Winner:

Game 964/1000 (Winner: white) in 81 turns
Game 965/1000 (Winner: black) in 91 turns
Game 966/1000 (Winner: black) in 244 turns
Game 967/1000 (Winner: white) in 86 turns
Game 968/1000 (Winner: white) in 96 turns
Game 969/1000 (Winner: black) in 107 turns
Game 970/1000 (Winner: black) in 109 turns
Game 971/1000 (Winner: white) in 90 turns
Game 972/1000 (Winner: white) in 68 turns
Game 973/1000 (Winner: white) in 79 turns
Game 974/1000 (Winner: white) in 115 turns
Game 975/1000 (Winner: black) in 117 turns
Game 976/1000 (Winner: black) in 69 turns
Game 977/1000 (Winner: white) in 169 turns
Game 978/1000 (Winner: white) in 124 turns
Game 979/1000 (Winner: black) in 65 turns
Game 980/1000 (Winner: white) in 135 turns
Game 981/1000 (Winner: black) in 224 turns
Game 982/1000 (Winner: black) in 79 turns
Game 983/1000 (Winner: black) in 87 turns
Game 984/1000 (Winner: black) in 123 turns
Game 985/1000 (Winner: white) in 77 turns
Game 986/1000 (Winner: white) in 60 turns
Game 987/1000 (Winner: b

[Episode 86] TD-Gammon0.0 (black) vs Random (white) 85:2 of 87 games (97.70%)
[Episode 87] TD-Gammon0.0 (black) vs Random (white) 86:2 of 88 games (97.73%)
[Episode 88] TD-Gammon0.0 (black) vs Random (white) 87:2 of 89 games (97.75%)
[Episode 89] TD-Gammon0.0 (black) vs Random (white) 88:2 of 90 games (97.78%)
[Episode 90] TD-Gammon0.0 (black) vs Random (white) 89:2 of 91 games (97.80%)
[Episode 91] TD-Gammon0.0 (black) vs Random (white) 90:2 of 92 games (97.83%)
[Episode 92] TD-Gammon0.0 (black) vs Random (white) 91:2 of 93 games (97.85%)
[Episode 93] TD-Gammon0.0 (black) vs Random (white) 92:2 of 94 games (97.87%)
[Episode 94] TD-Gammon0.0 (black) vs Random (white) 93:2 of 95 games (97.89%)
[Episode 95] TD-Gammon0.0 (black) vs Random (white) 94:2 of 96 games (97.92%)
[Episode 96] TD-Gammon0.0 (black) vs Random (white) 95:2 of 97 games (97.94%)
[Episode 97] TD-Gammon0.0 (black) vs Random (white) 96:2 of 98 games (97.96%)
[Episode 98] TD-Gammon0.0 (black) vs Random (white) 97:2 of 99 g

        4    0.002    0.001    0.002    0.001 {method 'MergeFrom' of 'google.protobuf.pyext._message.CMessage' objects}
       14    0.002    0.000    0.002    0.000 {method 'SerializeToString' of 'google.protobuf.pyext._message.CMessage' objects}
        1    0.000    0.000    0.000    0.000 {method '__subclasshook__' of 'object' objects}
     3272    0.003    0.000    0.003    0.000 {method 'acquire' of '_thread.lock' objects}
      138    0.000    0.000    0.000    0.000 {method 'add' of 'set' objects}
     3272    0.001    0.000    0.001    0.000 {method 'append' of 'collections.deque' objects}
       13    0.000    0.000    0.000    0.000 {method 'append' of 'google.protobuf.pyext._message.RepeatedScalarContainer' objects}
 22675285    2.676    0.000    2.676    0.000 {method 'append' of 'list' objects}
   238191    0.036    0.000    0.036    0.000 {method 'bit_length' of 'int' objects}
        1    0.000    0.000    0.000    0.000 {method 'close' of '_io.StringIO' objects}
      

In [223]:
#Lässt das Modell gegen das Modell aus dem github repo auf der repo Spiel Engine antreten 
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = OwnModel(sess, own_checkpoint_path, restore=True)
    graph2 = tf.Graph()
    sess2 = tf.Session(graph=graph2)
    with sess2.as_default(), graph2.as_default():
        model2 = Model(sess2, model_path, summary_path, checkpoint_path, restore=True)
        model.battle(enemyAgent=TDAgent(Game.TOKENS[1], model2), episodes=1000)

Restoring checkpoint: OWNcheckpoints/checkpoint 2333955.ckpt-2333955
INFO:tensorflow:Restoring parameters from OWNcheckpoints/checkpoint 2333955.ckpt-2333955
INFO:tensorflow:Summary name layer1/weight:0 is illegal; using layer1/weight_0 instead.
INFO:tensorflow:Summary name layer1/weight:0/gradients/grad is illegal; using layer1/weight_0/gradients/grad instead.
INFO:tensorflow:Summary name layer1/bias:0 is illegal; using layer1/bias_0 instead.
INFO:tensorflow:Summary name layer1/bias:0/gradients/grad is illegal; using layer1/bias_0/gradients/grad instead.
INFO:tensorflow:Summary name layer2/weight:0 is illegal; using layer2/weight_0 instead.
INFO:tensorflow:Summary name layer2/weight:0/gradients/grad is illegal; using layer2/weight_0/gradients/grad instead.
INFO:tensorflow:Summary name layer2/bias:0 is illegal; using layer2/bias_0 instead.
INFO:tensorflow:Summary name layer2/bias:0/gradients/grad is illegal; using layer2/bias_0/gradients/grad instead.
INFO:tensorflow:Summary name layer

[Episode 87] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:88 of 88 games (0.00%)
[Episode 88] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:89 of 89 games (0.00%)
[Episode 89] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:90 of 90 games (0.00%)
[Episode 90] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:91 of 91 games (0.00%)
[Episode 91] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:92 of 92 games (0.00%)
[Episode 92] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:93 of 93 games (0.00%)
[Episode 93] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:94 of 94 games (0.00%)
[Episode 94] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:95 of 95 games (0.00%)
[Episode 95] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:96 of 96 games (0.00%)
[Episode 96] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:97 of 97 games (0.00%)
[Episode 97] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:98 of 98 games (0.00%)
[Episode 98] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:99 of 99 games (0.00%)
[Episode 99] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:100 of 100 games (0.00%)
[Episode 100] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:101 of 101 g

[Episode 197] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:198 of 198 games (0.00%)
[Episode 198] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:199 of 199 games (0.00%)
[Episode 199] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:200 of 200 games (0.00%)
[Episode 200] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:201 of 201 games (0.00%)
[Episode 201] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:202 of 202 games (0.00%)
[Episode 202] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:203 of 203 games (0.00%)
[Episode 203] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:204 of 204 games (0.00%)
[Episode 204] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:205 of 205 games (0.00%)
[Episode 205] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:206 of 206 games (0.00%)
[Episode 206] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:207 of 207 games (0.00%)
[Episode 207] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:208 of 208 games (0.00%)
[Episode 208] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:209 of 209 games (0.00%)
[Episode 209] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:210 of 210 games (0.00%)
[Episode 210] TD-Gammon0.

[Episode 307] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:308 of 308 games (0.00%)
[Episode 308] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:309 of 309 games (0.00%)
[Episode 309] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:310 of 310 games (0.00%)
[Episode 310] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:311 of 311 games (0.00%)
[Episode 311] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:312 of 312 games (0.00%)
[Episode 312] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:313 of 313 games (0.00%)
[Episode 313] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:314 of 314 games (0.00%)
[Episode 314] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:315 of 315 games (0.00%)
[Episode 315] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:316 of 316 games (0.00%)
[Episode 316] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:317 of 317 games (0.00%)
[Episode 317] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:318 of 318 games (0.00%)
[Episode 318] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:319 of 319 games (0.00%)
[Episode 319] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:320 of 320 games (0.00%)
[Episode 320] TD-Gammon0.

[Episode 417] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:418 of 418 games (0.00%)
[Episode 418] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:419 of 419 games (0.00%)
[Episode 419] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:420 of 420 games (0.00%)
[Episode 420] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:421 of 421 games (0.00%)
[Episode 421] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:422 of 422 games (0.00%)
[Episode 422] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:423 of 423 games (0.00%)
[Episode 423] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:424 of 424 games (0.00%)
[Episode 424] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:425 of 425 games (0.00%)
[Episode 425] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:426 of 426 games (0.00%)
[Episode 426] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:427 of 427 games (0.00%)
[Episode 427] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:428 of 428 games (0.00%)
[Episode 428] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:429 of 429 games (0.00%)
[Episode 429] TD-Gammon0.0 (x) vs TD-Gammon (o) 0:430 of 430 games (0.00%)
[Episode 430] TD-Gammon0.

KeyboardInterrupt: 

Leider scheint der Unterschied der Spielengine dafür zu sorgen, dass mein Modell nur verliert....

In [7]:
"""
Ein verbesserter TDAgent der nicht nur 1-ply sondern auch 2-ply untersucht!
Zwar nicht vollständig 2-ply da die schiere Menge an möglichen Spielzügen zu lange braucht um in realistischer Zeit
vernünftige entscheidungen zu treffen. Stattdessen werden tries viele mögliche Würfe untersucht (3 hat eine angenehme 
Geschwindigkeit) und das Mittel aus der Güte der Gegnerzüge gezogen.
Daraufhin wird der Zug gewählt der den meisten Eigennutzen und den wenigsten Gegnernutzen hat.
"""

class TDAgent2Ply(object):

    def __init__(self, player, model):
        self.player = player
        self.model = model
        self.name = 'TD-Gammon2Ply'

    def get_action(self, actions, game):
        start = time.time()
        #Alle gültigen Züge nach den besten Folgezügen durchsuchen
        v_max = 0
        a_best = None
        delta = 0
        w_action = None
        for a in actions:
            #Eigenen Zug bewerten
            old_state = game.execute_moves(a, self.player)
            features = game.extractFeatures(self.player)
            v = self.model.get_output(features)
            v = 1. - v if self.player == game.players[0] else v
            if v > v_max:
                v_max = v
                a_best = a
            #Lookahead: Schaue dir ein paar Gegnerzüge an
            tries = 3
            opp_score = 0
            for i in range(tries):
                opp_score += self.evaluateOpponent(game)
            mean_score = opp_score / tries
            #Wähle den Zug der dir am meisten und dem Gegner am wenigsten bringt
            d = v - mean_score
            if d > delta:
                delta = d
                w_action = a
            #Zug rückgängig machen
            game.reset_to_state(old_state)   
        #Je nachdem ob etwas gefunden wurde zurückgeben
        #print("Zugezeit:", time.time()-start)
        w_action = a_best if not w_action else w_action
        return w_action
    
    def evaluateOpponent(self, game):
        player = game.get_opponent(self.player)
        #EINEN möglichen Wurf und ALLE Züge des Gegners betrachten
        roll = (random.randint(1,6), random.randint(1,6))
        actions = game.get_moves(roll, player)
        res = [self.getScore(a, game) for a in actions]
        #Der Gegner wird auch immer seinen besten Zug wählen
        if res:
            return max(res)
        else:
            return 1
    
    def getScore(self, action, game):
        player = game.get_opponent(self.player)
        #Zug ausführen
        old_state = game.execute_moves(action, player)
        features = game.extractFeatures(player)
        #Bewerten
        v = self.model.get_output(features)
        v = 1. - v if player == game.players[0] else v
        #Zug Rückgängig machen
        game.reset_to_state(old_state)
        #Ausgeben
        return v

In [234]:
#Diese Zelle lässt den normalen TDAgent gegen die 2ply Version antreten mit dem gleiche zugrunde liegenden Modell
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = OwnModel(sess, own_checkpoint_path, restore=True)
    model.test(enemyAgent=TDAgent2Ply('white', model), episodes=100)

Restoring checkpoint: own_checkpoints/checkpoint 351661.ckpt-351661
INFO:tensorflow:Restoring parameters from own_checkpoints/checkpoint 351661.ckpt-351661
[Episode 0] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 1:0 of 1 games (100.00%)
[Episode 1] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 1:1 of 2 games (50.00%)
[Episode 2] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 1:2 of 3 games (33.33%)
[Episode 3] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 2:2 of 4 games (50.00%)
[Episode 4] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 2:3 of 5 games (40.00%)
[Episode 5] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 3:3 of 6 games (50.00%)
[Episode 6] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 4:3 of 7 games (57.14%)
[Episode 7] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 5:3 of 8 games (62.50%)
[Episode 8] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 6:3 of 9 games (66.67%)
[Episode 9] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 6:4 of 10 games (60.00%)
[Episode 10] TD-Gammon

[Episode 95] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 66:30 of 96 games (68.75%)
[Episode 96] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 66:31 of 97 games (68.04%)
[Episode 97] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 67:31 of 98 games (68.37%)
[Episode 98] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 68:31 of 99 games (68.69%)
[Episode 99] TD-Gammon0.0 (black) vs TD-Gammon2Ply (white) 69:31 of 100 games (69.00%)


Dafür, dass die Rechenzeit vervielfacht wird ist der Nutzen ziemlich enttäuschend. 
(Oder der Agent ist fehlerhaft)

In [4]:
"""
Basierend auf "Monte-Carlo tree search in backgammon", Van Lishout, Chaslot und Uiterwijk, 2007

Schneller, besser kommentiert, aber auch leider länger als die andere Implementierung
"""
class OwnGame:
    
    def __init__(self):
        #Spielbrett. Index ist Position auf dem Spielfeld und der Wert die Anzahl der Steine auf dem Feld
        #Diese Zahl ist positiv für schwarze und negativ für Weiße Steine
        self.points = [0] * 24
        #Layout "0-2-o,5-5-x,7-3-x,11-5-o,12-5-x,16-3-o,18-5-o,23-2-x"
        self.points[0] = 2
        self.points[5] = -5
        self.points[7] = -3
        self.points[11] = 5
        self.points[12] = -5
        self.points[16] = 3
        self.points[18] = 5
        self.points[23] = -2
        #Steinpositionen
        self.refresh_piece_positions()
        #Steine die auf der Bar sind
        self.black_taken = 0
        self.white_taken = 0
        self.players = ["black", "white"]
        self.turns = 0
        
    # Methode die exakt die 198 Features liefert die in TD-Gammon 0.0 benutzt wurden
    # Nach "Reinforcement Learning: An Introduction", Sutton & Barto, 2017
    def extractFeatures(self, player):
        points = self.points
        # 196 Features kodieren den Zustand der Spielfelder, 98 für jeden Spieler
        # 0,1,2,3,4,5 Steine werden kodiert als
        # 0000, 1000, 1100, 1110, 1110.5, 1111
        # (4. Bit = (n-3)/2)
        features = []
        #Weiße Steine codieren
        whites = 0
        for point in points:
            point = -point
            if point > 0:
                whites += point
                features += self.encodePoint(point)
            else:
                features += [0.,0.,0.,0.]
        #Weiße Steine auf der "Bar", n/2
        features.append(self.white_taken/2.)
        #Weiße Steine die bereits aus dem Spiel sind, n/15
        off = 15 - whites + self.white_taken
        features.append(off/15.)
        #Schwarze Steine codieren
        blacks = 0
        for point in points:
            if point > 0:
                blacks += point
                features += self.encodePoint(point)
            else:
                features += [0.,0.,0.,0.]
        #Schwarze Steine auf der "Bar", n/2
        features.append(self.black_taken/2.)
        #Schwarze Steine die bereits aus dem Spiel sind, n/15
        off = 15 - blacks + self.black_taken
        features.append(off/15.)
        # Zwei Features für den derzeitigen Spieler
        if player == self.players[0]:
            features += [1., 0.]
        else:
            features += [0., 1.]
        return np.array(features).reshape(1, -1)
    
    def encodePoint(self, point):
        if point == 0:
            return [0.,0.,0.,0.]
        elif point == 1:
            return [1.,0.,0.,0.]
        elif point == 2:
            return [1.,1.,0.,0.]
        elif point == 3:
            return [1.,1.,1.,0.]
        else:
            return [1.,1.,1.,(point-3)/2.]
        
    def play(self, player, debug=False):
        #Wer anfängt ist zufällig
        player_num = random.randint(0, 1)
        #Solange spielen bis es einen Gewinner gibt
        while not self.get_winner():
            #Zug ausführen
            self.next_step(player[player_num], player_num, debug=debug)
            #Der andere Spieler ist dran
            player_num = (player_num + 1) % 2
        #Siegesstats ausgeben
        #self.print_game_state()
        return self.get_winner()
    
    #Spielt ein zufälliges Spiel und gibt den Gewinner zurück
    def play_random(self, start_player, debug=False):
        state = self.get_state()
        players = [RandomAgent(self.players[0]), RandomAgent(self.players[1])]
        player_num = 0 if start_player == self.players[0] else 1
        while not self.get_winner():
            #Zug ausführen
            self.next_step(players[player_num], player_num, debug=debug)
            #Der andere Spieler ist dran
            player_num = (player_num + 1) % 2
        winner = self.get_winner()
        #Zurücksetzen
        self.reset_to_state(state)
        return winner
    
    def next_step(self, player, player_num, debug=False):
        self.turns += 1
        #Würfeln
        roll = (random.randint(1,6), random.randint(1,6))
        #Züge berechnen
        moves = self.get_moves(roll, self.players[player_num])
        #Spieler fragen welche der Züge er gerne ausführen möchte
        move = player.get_action(moves, self) if moves else None
        #Zug ausführen falls es möglich ist 
        if move:
            #Einzelne Unterzüge ausführen
            if debug:
                print(move)
            self.execute_moves(move, self.players[player_num])
        #Debuggen
        if debug:
            print("Current Player:", self.players[player_num])
            print("Moves:", moves)
            print("Roll:", roll, "| Move:", move)
            self.print_game_state()
            print()
            
    def execute_moves(self, moves, player):
        #Alten Zustand merken
        old_state = self.get_state()
        #Unterzüge ausführen
        for m in moves:
            self.execute_move(m, player)
        #Alte position merken
        return old_state
            
    #Führt einen Zug aus und gibt die vorherige Spielposition zurück
    def execute_move(self, move, player):
        if move != (0,0):
            piece = 1 if player == self.players[0] else - 1
            #Stein von der alten Position nehmen, falls nicht auf der Bar
            if move[0] != "bar":
                self.points[move[0]] -= piece
            elif player == self.players[0]:
                self.black_taken -= 1
            else:
                self.white_taken -= 1
            #Stein auf die gewünschte Stelle setzen, falls noch auf dem Spielfeld
            if move[1] >= 0 and move[1] < len(self.points):
                #Falls dort bereits ein Gegnerstein war wird er auf die Bar gelegt
                if player == self.players[0] and self.points[move[1]] == -1:
                    self.points[move[1]] = 0
                    self.white_taken += 1
                elif player == self.players[1] and self.points[move[1]] == 1:
                    self.points[move[1]] = 0
                    self.black_taken += 1
                #Stein platzieren
                self.points[move[1]] += piece
            #Positionen der schwarzen und weißen Steine aktualisieren
            self.refresh_piece_positions()
            
    def get_state(self):
        return (self.points[:], self.black_taken, self.white_taken)
    
    #Setzt das Spiel auf die gegbene Spielposition (zurück)
    def reset_to_state(self, state):
        self.points = state[0]
        self.black_taken = state[1]
        self.white_taken = state[2]
        #Positionen der schwarzen und weißen Steine aktualisieren
        self.refresh_piece_positions()

    #Aktualisiert die Listen mit den Position der Steine
    def refresh_piece_positions(self):
        #Positionen der schwarzen Steine
        self.black_checkers = [i for i in range(24) if self.points[i] > 0]
        #Positionen der weißen Steine
        self.white_checkers = sorted([i for i in range(24) if self.points[i] < 0], reverse=True)
    
    def get_moves(self, roll, player):
        #Pasch?
        if roll[0] == roll[1]:
            return self.get_quad_moves(roll[0], player)
        #Hat der Spieler Steine die er erst wieder ins Spiel bringen muss?
        if self.has_bar_pieces(player):
            return self.get_bar_to_board_moves(roll, player)
        #Sonstige Züge finden
        else:
            return self.generate_moves(roll, player)

    def get_bar_to_board_moves(self, roll, player):
        moves = []
        #Sind die Heimfelder blockiert die die Würfel anzeigen?
        pos0 = roll[0] - 1 if player == self.players[0] else len(self.points) - roll[0]
        pos1 = roll[1] - 1 if player == self.players[0] else len(self.points) - roll[1]
        val1 = self.is_target_valid(pos0, player)
        val2 = self.is_target_valid(pos1, player)
        taken = self.black_taken if player == self.players[0] else self.white_taken
        #Falls beide Würfel genutzt werden können müssen sie genutzt werden
        if taken > 1 and val1 and val2:
            moves.append((("bar", pos0), ("bar", pos1)))
        else:
            #Falls nicht möglich, andere Züge für den zweiten würfel finden
            if val1:
                bar_move = ("bar", pos0)
                singles = self.generate_single_move(bar_move, roll[1], player)
                moves += [(bar_move, s) for s in singles]
            if val2:
                bar_move = ("bar", pos1)
                singles = self.generate_single_move(bar_move, roll[0], player)
                moves += [(bar_move, s) for s in singles]
        return moves

    def generate_moves(self, roll, player):
        valid = self.is_target_valid
        board = self.points
        #Alle zweier Kombinationen aus den derzeitigen Positionen ermitteln
        chk = self.black_checkers if player == self.players[0] else self.white_checkers
        comb = list(itertools.combinations(chk, 2))
        comb += [(a,a) for a in chk if board[a] > 1 or board[a] < -1]
        #Züge suchen
        moves = []
        #Schwarz geht die Zahlen hoch, Weiß runter
        if player == self.players[1]:
            roll = (-roll[0], -roll[1])
        #Jeden Zug prüfen
        for (a,b) in comb:
            #Zwei Steine Bewegen
            a0 = valid(a + roll[0], player)
            a1 = valid(a + roll[1], player)
            if a0 and valid(b + roll[1], player):
                moves.append(((a, a + roll[0]), (b, b + roll[1])))
            if a1 and valid(b + roll[0], player) and not (a==b and a0):
                moves.append(((a, a + roll[1]), (b, b + roll[0])))
            #Ein Stein bewegen
            farpos = a + roll[0] + roll[1]
            if a == b and farpos >= 0 and farpos < len(self.points) and valid(farpos, player):
                if a0:
                    moves.append(((a, a + roll[0]), (a + roll[0], farpos)))
                elif a1:
                    moves.append(((a, a + roll[1]), (a + roll[1], farpos)))
        #Falls man keine zwei Züge generieren kann, schauen ob man vielleicht einen Einzelnen schafft
        if moves == []:
            m1 = self.generate_single_move(prev_move=None, dice=roll[0], player=player)
            m2 = self.generate_single_move(prev_move=None, dice=roll[1], player=player)
            m1.extend(m2)
            moves += [((0,0), m) for m in m1]
        #Zurückgeben
        return moves
    
    def generate_single_move(self, prev_move, dice, player):
        chk = self.black_checkers if player == self.players[0] else self.white_checkers
        if player == self.players[1] and dice > 0:
            dice = -dice
        #Alle normalen Züge 
        moves = [(x, x + dice) for x in chk if self.is_target_valid(x + dice, player)]
        #kann man den Stein direkt weiter ziehen?
        if prev_move and self.is_target_valid(prev_move[1] + dice, player):
            moves.append((prev_move[1], prev_move[1] + dice))
        return moves
    
    def generate_double_move(self, prev_move, dice, player):
        chk = self.black_checkers if player == self.players[0] else self.white_checkers
        if player == self.players[1] and dice > 0:
            dice = -dice
        #Einzelzüge besorgen
        s_moves = self.generate_single_move(prev_move, dice, player)
        #Weiteren Zug anhängen
        moves = [((a,b),(b,b+dice)) for (a,b) in s_moves if self.is_target_valid(b+dice, player)]
        for x in chk:
            for y in s_moves:
                if self.is_target_valid(x+dice, player):
                    if ((x,x+dice) == y and self.points[x] > 2) or (x,x+dice) != y:
                        moves.append((y, (x,x+dice)))
        return moves
            
    def generate_triple_move(self, prev_move, dice, player):
        chk = self.black_checkers if player == self.players[0] else self.white_checkers
        if player == self.players[1] and dice > 0:
            dice = -dice
        #Doppelzüge besorgen
        d_moves = self.generate_double_move(prev_move, dice, player)
        #Weiteren Zug anhängen
        moves = [((a,b), (c,d), (d, d+dice)) for ((a,b), (c,d)) in d_moves if self.is_target_valid(d+dice, player)]
        for x in chk:
            for (y,z) in d_moves:
                if self.is_target_valid(x+dice, player):
                    if (((x,x+dice) == y or (x,x+dice) == z) and self.points[x] > 2) or ((x,x+dice) != y and (x,x+dice) != z):
                        moves.append((y, z, (x,x+dice)))
        return moves
            
    #Bei einem Pasch müssen 4 Züge ausgeführt werden
    def get_quad_moves(self, dice, player):
        #Bis zu 4 Steine von der Bar bewegen
        if self.has_bar_pieces(player):
            return self.get_quad_bar_to_board_moves(dice, player)
        #4 Züge finden
        else:
            return self.generate_quad_moves(dice, player)
        
    def get_quad_bar_to_board_moves(self, dice, player):
        moves = []
        #Ist das von den Würfeln gezeigt Heimfeld blockiert?
        pos = dice - 1 if player == self.players[0] else len(self.points) - dice
        valid = self.is_target_valid(pos, player)
        taken = self.black_taken if player == self.players[0] else self.white_taken
        #Falls alle Würfel genutzt werden können müssen sie genutzt werden
        if valid:
            bar_move = ("bar", pos)
            if taken >= 4:
                moves.append((bar_move, bar_move, bar_move, bar_move))
            elif taken == 3:
                singles = self.generate_single_move(bar_move, dice, player)
                moves += [(bar_move, bar_move, bar_move, s) for s in singles]
            elif taken == 2:
                doubles = self.generate_double_move(bar_move, dice, player)
                moves += [(bar_move, bar_move, d1, d2) for (d1, d2) in doubles]
            else:
                triples = self.generate_triple_move(bar_move, dice, player)
                moves += [(bar_move, t1, t2, t3) for (t1, t2, t3) in triples]
        return moves
  
    """
        Kombinationsmöglichkeiten:
        1. quad
        2. triple + single
        3. double + double
        4. double + single + single
        5. single + single + single + single
    """
    def generate_quad_moves(self, dice, player):
        #Optimieren
        board = self.points
        valid = self.is_target_valid
        #Weiß läuft entegengesetzt
        if player == self.players[1] and dice > 0:
            dice = -dice
        #Alle zweier Kombinationen aus den derzeitigen Positionen ermitteln
        chk = self.black_checkers if player == self.players[0] else self.white_checkers
        #Alle Positionen mit mindestens einem Stein
        single = chk[:]
        #Alle Positionen mit mindestens zwei Steinen
        double = [x for x in chk if abs(board[x]) >= 2]
        #Alle Positionen mit mindestens drei Steinen
        triple = [x for x in chk if abs(board[x]) >= 3]
        #Alle Positionen mit mindestens vier Steinen
        quad = [x for x in chk if abs(board[x]) >= 4]
        #Gültige Zielpunkte sammeln
        valid_dict = {}
        for s in single:
            for i in range(4):
                target = s+dice*(i+1)
                if target not in valid_dict:
                    if i == 0:
                        valid_dict[target] = valid(target, player)
                    else:
                        valid_dict[target] = valid(target, player) and target >= 0 and target < len(self.points)
                        
        moves = []
        
        #Quads, 1. Kombination
        for q in quad:
            if valid_dict[q+dice]:
                moves.append(((q, q+dice),(q, q+dice),(q, q+dice),(q, q+dice)))
                
        #Triples
        for t in triple:
            #2. Kombination
            for s in single:
                if t != s and valid_dict[t+dice] and valid_dict[s+dice]:
                    moves.append(((t, t+dice),(t, t+dice),(t, t+dice),(s, s+dice)))
            #Folgezüge für triples
            if valid_dict[t+dice] and valid_dict[t+dice*2]:
                moves.append(((t, t+dice),(t, t+dice),(t, t+dice),(t+dice, t+dice*2)))   
                
        #Doubles
        for d in double:
            d1 = valid_dict[d+dice]
            d2 = valid_dict[d+dice*2]
            d3 = valid_dict[d+dice*3]
            #3. Kombination
            for ds in double:
                #Keine Doppelten bitte
                if ds > d:
                    if d1 and valid_dict[ds+dice]:
                        moves.append(((d, d+dice),(d, d+dice),(ds, ds+dice),(ds, ds+dice)))
            #4. Kombination
            for s1 in single:
                for s2 in single:
                    if s2 > s1 and d != s1 and d != s2:
                        if valid_dict[d+dice] and valid_dict[s1+dice] and valid_dict[s2+dice]:
                            moves.append(((d, d+dice),(d, d+dice),(s1, s1+dice),(s2, s2+dice)))
                #Doppelzug gefolgt von einem Folgezug und einem single
                if d1 and d2 and d != s1 and valid_dict[s1+dice]:
                        moves.append(((d, d+dice),(d, d+dice),(d+dice, d+dice*2),(s1, s1+dice)))
            #Folgezüge für doubles
            #Jeweils zwei Züge mit zwei Steinen
            if d1 and d2:
                moves.append(((d, d+dice),(d, d+dice),(d+dice, d+dice*2),(d+dice, d+dice*2)))
            #Ein double gefolgt von einem doppelten Folgezug
            if d1 and d2 and d3:
                moves.append(((d, d+dice),(d, d+dice),(d+dice, d+dice*2),(d+dice*2, d+dice*3)))

        #Singles
        for s1 in single:
            sv1 = valid_dict[s1+dice]
            sv2 = valid_dict[s1+dice*2]
            sv3 = valid_dict[s1+dice*3]
            sv4 = valid_dict[s1+dice*4]
            for s2 in single:
                sec1 = valid_dict[s2+dice]
                sec2 = valid_dict[s2+dice*2]
                sec3 = valid_dict[s2+dice*3]
                for s3 in single:
                    for s4 in single:
                        #5. Kombination
                        if s4 > s3 > s2 > s1:
                            if sv1 and valid_dict[s2+dice] and valid_dict[s3+dice] and valid_dict[s4+dice]:
                                moves.append(((s1, s1+dice),(s2, s2+dice),(s3, s3+dice),(s4, s4+dice)))
                    #Züge mit mindestens drei Steinen
                    if s2 > s1 and s3 != s2 and s3 != s1:
                        #Zwei Einzelzüge gefolgt von einem Folgezug
                        if sv1 and sec1 and valid_dict[s3+dice] and valid_dict[s3+dice*2]:
                            moves.append(((s1, s1+dice),(s2, s2+dice),(s3, s3+dice),(s3+dice, s3+dice*2)))
                
            #Folgezüge für singles
                if s2 > s1:
                    #Ein Zug mit dem ersten Stein und drei mit dem zweiten
                    if sv1 and sec1 and sec2 and sec3:
                        moves.append(((s1, s1+dice),(s2, s2+dice),(s2+dice, s2+dice*2),(s2+dice*2, s2+dice*3)))
                    #Zwei Züge mit dem ersten und zwei mit dem zweiten
                    if sv1 and sv2 and sec1 and sec2:
                        moves.append(((s1, s1+dice),(s1+dice, s1+dice*2),(s2, s2+dice),(s2+dice, s2+dice*2)))
                    #Drei Züge mit dem ersten Stein und einen weiteren mit dem zweiten
                    if sv1 and sv2 and sv3 and sec1:
                        moves.append(((s1, s1+dice),(s1+dice, s1+dice*2),(s1+dice*2, s1+dice*3),(s2, s2+dice)))
            #Zwei Züge mit dem ersten Stein gefolgt von einem double
            for d in double:
                if d != s1 and sv1 and sv2 and valid_dict[d+dice]:
                    moves.append(((s1, s1+dice),(s1+dice, s1+dice*2),(d, d+dice),(d, d+dice)))
                    
            #Vier Züge mit einem einzigen Stein
            if sv1 and sv2 and sv3 and sv4:
                moves.append(((s1, s1+dice),(s1+dice, s1+dice*2),(s1+dice*2, s1+dice*3),(s1+dice*3, s1+dice*4)))

        #Tupel sortieren um Doppelte zu finden und zu löschen
        #Erhöht zwar hier den Rechenaufwand, aber reduziert die Anzahl der Züge erheblich!
        if player == self.players[0]:
            return set([tuple(sorted(x)) for x in moves])
        else:
            return set([tuple(sorted(x, reverse=True)) for x in moves])

    #Prüft ob das angegeben Ziel gültig ist
    def is_target_valid(self, target, player):
        #Landen wir jenseis des Spielbretts?
        if target < 0 or target >= len(self.points):
            if self.can_offboard(player):
                return True
            else:
                return False
        #Prüfen ob das Ziel blockiert ist (2 oder mehr Gegnersteine vorhanden)
        if player == self.players[0] and self.points[target] > -2:
            return True
        elif player == self.players[1] and self.points[target] < 2:
            return True
        else:
            return False
            
    #Hat der Spieler Steine die vom Gegner rausgeworfen wurden?
    def has_bar_pieces(self, player):
        if player == self.players[0] and self.black_taken > 0:
            return True
        elif player == self.players[1] and self.white_taken > 0:
            return True
        else:
            return False
    
    #Hat der Spieler alle seine Steine in seiner Homezone?
    def can_offboard(self, player):
        if player == self.players[0] and self.black_taken == 0 and self.black_checkers[0] > 18:
            return True
        elif player == self.players[1] and self.white_taken == 0 and self.white_checkers[0] < 7:
            return True
        else:
            return False
        
    #Gibt den Gewinner zurück falls es einen gibt
    def get_winner(self):
        if self.black_checkers == [] and self.black_taken == 0:
            return self.players[0]
        elif self.white_checkers == [] and self.white_taken == 0:
            return self.players[1]
        else:
            return None
        
    def get_opponent(self, player):
        return self.players[0] if player == self.players[1] else self.players[1]
    
    def Clone(self):
        g = OwnGame()
        g.reset_to_state(self.get_state())
        return g
        
    def print_game_state(self):
        print("GameState:", self.points, "|", self.black_checkers, "|", self.white_checkers)
        bl = sum([self.points[i] for i in range(len(self.points)) if self.points[i] > 0])
        wt = sum([self.points[i] for i in range(len(self.points)) if self.points[i] < 0])
        print("Black/White:", bl, "/", wt, "Bar:", self.black_taken, "/", self.white_taken)

In [26]:
#Ständiges vergleichen und Zeitmessungen waren nötig um ein (hoffentlich) besseres Backgammon Programm zu implementieren
g = OwnGame()
gAlt = Game.new()

roll = (2,2)
player = 0

start = time.time()
for i in range(1000):
    actions1 = sorted(g.get_moves(roll, g.players[player]))
    #actions1 = sorted(g.generate_triple_move((1,3), 2, g.players[player]))
mitte = time.time()
for i in range(1000):
    actions2 = sorted(gAlt.get_actions(roll, gAlt.players[1]))
ende = time.time()
print(actions2, len(actions2))
print(actions1, len(actions1))
print("Neu:", mitte-start, "| Alt:", ende-mitte)
#print("Es fehlen:", [(a,b) for (a,b) in actions2 if (a,b) not in actions1 and (b,a) not in actions1])
#print("Dafür haben wir:", [(a,b) for (a,b) in actions1 if (a,b) not in actions2 and (b,a) not in actions2])
print("Es fehlen:", [x for x in actions2 if x not in actions1])
print("Dafür haben wir:", [x for x in actions1 if x not in actions2])

actions2_sorted = [tuple(sorted(x)) for x in actions2]
#print(actions2_sorted)
clean_actions2 = sorted(set(actions2_sorted))
print(clean_actions2, len(clean_actions2))

clean_actions1 = sorted(set([tuple(sorted(x)) for x in actions1]))
print("gesäubert", clean_actions1, len(clean_actions1))
print("duplikate", [x for x in actions1 if x not in clean_actions1])
print("Es fehlen:", [x for x in clean_actions2 if x not in clean_actions1])
print("Dafür haben wir:", [x for x in clean_actions1 if x not in clean_actions2])

[((0, 2), (0, 2), (2, 4), (2, 4)), ((0, 2), (0, 2), (2, 4), (4, 6)), ((0, 2), (0, 2), (2, 4), (11, 13)), ((0, 2), (0, 2), (2, 4), (16, 18)), ((0, 2), (0, 2), (2, 4), (18, 20)), ((0, 2), (0, 2), (11, 13), (2, 4)), ((0, 2), (0, 2), (11, 13), (11, 13)), ((0, 2), (0, 2), (11, 13), (13, 15)), ((0, 2), (0, 2), (11, 13), (16, 18)), ((0, 2), (0, 2), (11, 13), (18, 20)), ((0, 2), (0, 2), (16, 18), (2, 4)), ((0, 2), (0, 2), (16, 18), (11, 13)), ((0, 2), (0, 2), (16, 18), (16, 18)), ((0, 2), (0, 2), (16, 18), (18, 20)), ((0, 2), (0, 2), (18, 20), (2, 4)), ((0, 2), (0, 2), (18, 20), (11, 13)), ((0, 2), (0, 2), (18, 20), (16, 18)), ((0, 2), (0, 2), (18, 20), (18, 20)), ((0, 2), (0, 2), (18, 20), (20, 22)), ((0, 2), (2, 4), (0, 2), (2, 4)), ((0, 2), (2, 4), (0, 2), (4, 6)), ((0, 2), (2, 4), (0, 2), (11, 13)), ((0, 2), (2, 4), (0, 2), (16, 18)), ((0, 2), (2, 4), (0, 2), (18, 20)), ((0, 2), (2, 4), (4, 6), (0, 2)), ((0, 2), (2, 4), (4, 6), (6, 8)), ((0, 2), (2, 4), (4, 6), (11, 13)), ((0, 2), (2, 4), 

In [ ]:
#import cProfile

game = OwnGame()
players = [RandomAgent(game.players[0]), RandomAgent(game.players[1])]

#cProfile.run('game.play(players)')

print(game.play(players, debug=True))
print(game.turns)

In [11]:
#Letzter Test: Alt gegen neu, wer schafft schneller 1000 random Spiele
#Sehr wichtiger Faktor für Monte Carlo tree search
start = time.time()
players = [RandomAgent('black'), RandomAgent('white')]
winners = {'black':0, 'white':0}
for episode in range(10000):
    game = OwnGame()
    winner = game.play(players, debug=False)
    winners[winner] += 1
    winners_total = sum(winners.values())
    print("[Episode %d] %s (%s) vs %s (%s) %d:%d of %d games (%.2f%%)" % (episode, \
        players[0].name, players[0].player, \
        players[1].name, players[1].player, \
        winners['black'], winners['white'], winners_total, \
        (winners['black'] / winners_total) * 100.0))
    
mid = time.time()
"""
players = [RandomAgent(Game.TOKENS[0]), RandomAgent(Game.TOKENS[1])]
winners = [0, 0]
for episode in range(1000):
    game = Game.new()

    winner = game.play(players, draw=False)
    winners[winner] += 1

    winners_total = sum(winners)
    print("[Episode %d] %s (%s) vs %s (%s) %d:%d of %d games (%.2f%%)" % (episode, \
        players[0].name, players[0].player, \
        players[1].name, players[1].player, \
        winners[0], winners[1], winners_total, \
        (winners[0] / winners_total) * 100.0))
"""
print("Neu:", mid-start, "Alt:", time.time()-mid)

[Episode 0] Random (black) vs Random (white) 1:0 of 1 games (100.00%)
[Episode 1] Random (black) vs Random (white) 2:0 of 2 games (100.00%)
[Episode 2] Random (black) vs Random (white) 2:1 of 3 games (66.67%)
[Episode 3] Random (black) vs Random (white) 3:1 of 4 games (75.00%)
[Episode 4] Random (black) vs Random (white) 3:2 of 5 games (60.00%)
[Episode 5] Random (black) vs Random (white) 4:2 of 6 games (66.67%)
[Episode 6] Random (black) vs Random (white) 5:2 of 7 games (71.43%)
[Episode 7] Random (black) vs Random (white) 6:2 of 8 games (75.00%)
[Episode 8] Random (black) vs Random (white) 6:3 of 9 games (66.67%)
[Episode 9] Random (black) vs Random (white) 7:3 of 10 games (70.00%)
[Episode 10] Random (black) vs Random (white) 8:3 of 11 games (72.73%)
[Episode 11] Random (black) vs Random (white) 8:4 of 12 games (66.67%)
[Episode 12] Random (black) vs Random (white) 8:5 of 13 games (61.54%)
[Episode 13] Random (black) vs Random (white) 8:6 of 14 games (57.14%)
[Episode 14] Random (bl

[Episode 114] Random (black) vs Random (white) 59:56 of 115 games (51.30%)
[Episode 115] Random (black) vs Random (white) 60:56 of 116 games (51.72%)
[Episode 116] Random (black) vs Random (white) 60:57 of 117 games (51.28%)
[Episode 117] Random (black) vs Random (white) 60:58 of 118 games (50.85%)
[Episode 118] Random (black) vs Random (white) 60:59 of 119 games (50.42%)
[Episode 119] Random (black) vs Random (white) 60:60 of 120 games (50.00%)
[Episode 120] Random (black) vs Random (white) 61:60 of 121 games (50.41%)
[Episode 121] Random (black) vs Random (white) 62:60 of 122 games (50.82%)
[Episode 122] Random (black) vs Random (white) 63:60 of 123 games (51.22%)
[Episode 123] Random (black) vs Random (white) 63:61 of 124 games (50.81%)
[Episode 124] Random (black) vs Random (white) 63:62 of 125 games (50.40%)
[Episode 125] Random (black) vs Random (white) 64:62 of 126 games (50.79%)
[Episode 126] Random (black) vs Random (white) 65:62 of 127 games (51.18%)
[Episode 127] Random (bla

[Episode 223] Random (black) vs Random (white) 107:117 of 224 games (47.77%)
[Episode 224] Random (black) vs Random (white) 107:118 of 225 games (47.56%)
[Episode 225] Random (black) vs Random (white) 108:118 of 226 games (47.79%)
[Episode 226] Random (black) vs Random (white) 108:119 of 227 games (47.58%)
[Episode 227] Random (black) vs Random (white) 108:120 of 228 games (47.37%)
[Episode 228] Random (black) vs Random (white) 109:120 of 229 games (47.60%)
[Episode 229] Random (black) vs Random (white) 110:120 of 230 games (47.83%)
[Episode 230] Random (black) vs Random (white) 111:120 of 231 games (48.05%)
[Episode 231] Random (black) vs Random (white) 112:120 of 232 games (48.28%)
[Episode 232] Random (black) vs Random (white) 112:121 of 233 games (48.07%)
[Episode 233] Random (black) vs Random (white) 112:122 of 234 games (47.86%)
[Episode 234] Random (black) vs Random (white) 112:123 of 235 games (47.66%)
[Episode 235] Random (black) vs Random (white) 112:124 of 236 games (47.46%)

[Episode 340] Random (black) vs Random (white) 166:175 of 341 games (48.68%)
[Episode 341] Random (black) vs Random (white) 166:176 of 342 games (48.54%)
[Episode 342] Random (black) vs Random (white) 167:176 of 343 games (48.69%)
[Episode 343] Random (black) vs Random (white) 168:176 of 344 games (48.84%)
[Episode 344] Random (black) vs Random (white) 168:177 of 345 games (48.70%)
[Episode 345] Random (black) vs Random (white) 168:178 of 346 games (48.55%)
[Episode 346] Random (black) vs Random (white) 169:178 of 347 games (48.70%)
[Episode 347] Random (black) vs Random (white) 170:178 of 348 games (48.85%)
[Episode 348] Random (black) vs Random (white) 171:178 of 349 games (49.00%)
[Episode 349] Random (black) vs Random (white) 171:179 of 350 games (48.86%)
[Episode 350] Random (black) vs Random (white) 172:179 of 351 games (49.00%)
[Episode 351] Random (black) vs Random (white) 172:180 of 352 games (48.86%)
[Episode 352] Random (black) vs Random (white) 173:180 of 353 games (49.01%)

[Episode 449] Random (black) vs Random (white) 218:232 of 450 games (48.44%)
[Episode 450] Random (black) vs Random (white) 218:233 of 451 games (48.34%)
[Episode 451] Random (black) vs Random (white) 219:233 of 452 games (48.45%)
[Episode 452] Random (black) vs Random (white) 219:234 of 453 games (48.34%)
[Episode 453] Random (black) vs Random (white) 219:235 of 454 games (48.24%)
[Episode 454] Random (black) vs Random (white) 220:235 of 455 games (48.35%)
[Episode 455] Random (black) vs Random (white) 221:235 of 456 games (48.46%)
[Episode 456] Random (black) vs Random (white) 222:235 of 457 games (48.58%)
[Episode 457] Random (black) vs Random (white) 223:235 of 458 games (48.69%)
[Episode 458] Random (black) vs Random (white) 223:236 of 459 games (48.58%)
[Episode 459] Random (black) vs Random (white) 224:236 of 460 games (48.70%)
[Episode 460] Random (black) vs Random (white) 225:236 of 461 games (48.81%)
[Episode 461] Random (black) vs Random (white) 225:237 of 462 games (48.70%)

...

[Episode 9460] Random (black) vs Random (white) 4547:4914 of 9461 games (48.06%)
[Episode 9461] Random (black) vs Random (white) 4548:4914 of 9462 games (48.07%)
[Episode 9462] Random (black) vs Random (white) 4548:4915 of 9463 games (48.06%)
[Episode 9463] Random (black) vs Random (white) 4548:4916 of 9464 games (48.06%)
[Episode 9464] Random (black) vs Random (white) 4549:4916 of 9465 games (48.06%)
[Episode 9465] Random (black) vs Random (white) 4549:4917 of 9466 games (48.06%)
[Episode 9466] Random (black) vs Random (white) 4549:4918 of 9467 games (48.05%)
[Episode 9467] Random (black) vs Random (white) 4550:4918 of 9468 games (48.06%)
[Episode 9468] Random (black) vs Random (white) 4550:4919 of 9469 games (48.05%)
[Episode 9469] Random (black) vs Random (white) 4550:4920 of 9470 games (48.05%)
[Episode 9470] Random (black) vs Random (white) 4551:4920 of 9471 games (48.05%)
[Episode 9471] Random (black) vs Random (white) 4551:4921 of 9472 games (48.05%)
[Episode 9472] Random (black

[Episode 9575] Random (black) vs Random (white) 4597:4979 of 9576 games (48.01%)
[Episode 9576] Random (black) vs Random (white) 4598:4979 of 9577 games (48.01%)
[Episode 9577] Random (black) vs Random (white) 4598:4980 of 9578 games (48.01%)
[Episode 9578] Random (black) vs Random (white) 4598:4981 of 9579 games (48.00%)
[Episode 9579] Random (black) vs Random (white) 4598:4982 of 9580 games (48.00%)
[Episode 9580] Random (black) vs Random (white) 4598:4983 of 9581 games (47.99%)
[Episode 9581] Random (black) vs Random (white) 4598:4984 of 9582 games (47.99%)
[Episode 9582] Random (black) vs Random (white) 4599:4984 of 9583 games (47.99%)
[Episode 9583] Random (black) vs Random (white) 4599:4985 of 9584 games (47.99%)
[Episode 9584] Random (black) vs Random (white) 4599:4986 of 9585 games (47.98%)
[Episode 9585] Random (black) vs Random (white) 4599:4987 of 9586 games (47.98%)
[Episode 9586] Random (black) vs Random (white) 4599:4988 of 9587 games (47.97%)
[Episode 9587] Random (black

[Episode 9679] Random (black) vs Random (white) 4649:5031 of 9680 games (48.03%)
[Episode 9680] Random (black) vs Random (white) 4649:5032 of 9681 games (48.02%)
[Episode 9681] Random (black) vs Random (white) 4649:5033 of 9682 games (48.02%)
[Episode 9682] Random (black) vs Random (white) 4650:5033 of 9683 games (48.02%)
[Episode 9683] Random (black) vs Random (white) 4651:5033 of 9684 games (48.03%)
[Episode 9684] Random (black) vs Random (white) 4651:5034 of 9685 games (48.02%)
[Episode 9685] Random (black) vs Random (white) 4651:5035 of 9686 games (48.02%)
[Episode 9686] Random (black) vs Random (white) 4651:5036 of 9687 games (48.01%)
[Episode 9687] Random (black) vs Random (white) 4651:5037 of 9688 games (48.01%)
[Episode 9688] Random (black) vs Random (white) 4652:5037 of 9689 games (48.01%)
[Episode 9689] Random (black) vs Random (white) 4653:5037 of 9690 games (48.02%)
[Episode 9690] Random (black) vs Random (white) 4653:5038 of 9691 games (48.01%)
[Episode 9691] Random (black

[Episode 9791] Random (black) vs Random (white) 4712:5080 of 9792 games (48.12%)
[Episode 9792] Random (black) vs Random (white) 4713:5080 of 9793 games (48.13%)
[Episode 9793] Random (black) vs Random (white) 4713:5081 of 9794 games (48.12%)
[Episode 9794] Random (black) vs Random (white) 4714:5081 of 9795 games (48.13%)
[Episode 9795] Random (black) vs Random (white) 4714:5082 of 9796 games (48.12%)
[Episode 9796] Random (black) vs Random (white) 4715:5082 of 9797 games (48.13%)
[Episode 9797] Random (black) vs Random (white) 4715:5083 of 9798 games (48.12%)
[Episode 9798] Random (black) vs Random (white) 4716:5083 of 9799 games (48.13%)
[Episode 9799] Random (black) vs Random (white) 4717:5083 of 9800 games (48.13%)
[Episode 9800] Random (black) vs Random (white) 4717:5084 of 9801 games (48.13%)
[Episode 9801] Random (black) vs Random (white) 4718:5084 of 9802 games (48.13%)
[Episode 9802] Random (black) vs Random (white) 4718:5085 of 9803 games (48.13%)
[Episode 9803] Random (black

[Episode 9895] Random (black) vs Random (white) 4760:5136 of 9896 games (48.10%)
[Episode 9896] Random (black) vs Random (white) 4760:5137 of 9897 games (48.10%)
[Episode 9897] Random (black) vs Random (white) 4760:5138 of 9898 games (48.09%)
[Episode 9898] Random (black) vs Random (white) 4760:5139 of 9899 games (48.09%)
[Episode 9899] Random (black) vs Random (white) 4760:5140 of 9900 games (48.08%)
[Episode 9900] Random (black) vs Random (white) 4760:5141 of 9901 games (48.08%)
[Episode 9901] Random (black) vs Random (white) 4761:5141 of 9902 games (48.08%)
[Episode 9902] Random (black) vs Random (white) 4761:5142 of 9903 games (48.08%)
[Episode 9903] Random (black) vs Random (white) 4761:5143 of 9904 games (48.07%)
[Episode 9904] Random (black) vs Random (white) 4762:5143 of 9905 games (48.08%)
[Episode 9905] Random (black) vs Random (white) 4762:5144 of 9906 games (48.07%)
[Episode 9906] Random (black) vs Random (white) 4763:5144 of 9907 games (48.08%)
[Episode 9907] Random (black

Die neue Engine ist ca. doppelt so schnell!

In [30]:
"""
Inspiriert von http://mcts.ai/code/python.html
"""

class Node:
    
    #Erzeugt einen neuen Knoten
    def __init__(self, move = None, parent = None, actions = None, player = None):
        self.move = move
        self.parentNode = parent
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = list(actions)
        self.playerJustMoved = player
        
    #Wendet die UCB1 Formel an um den besten Knoten zu finden
    def UCTSelectChild(self):
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + math.sqrt(2*math.log(self.visits)/c.visits))[-1]
        return s
    
    #Hängt einen neuen Knoten an den Baum mit diesem Knoten als Vaterknoten
    def AddChild(self, m, a, p):
        n = Node(move = m, parent = self, actions = a, player = p)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    #Updated die Statistik
    def Update(self, result):
        self.visits += 1
        self.wins += result

class TDMCAgent:
    
    def __init__(self, player, model):
        self.player = player
        self.model = model
        self.name = "TDMC"
        
    #Kombiniert Monte Carlo tree search mit dem Modell um noch besser zu entscheiden
    def get_action(self, actions, game):
        return self.UCT(actions, game, 50)

    def UCT(self, actions, rootgame, itermax):
        #Erstellt die Wurzel
        rootnode = Node(actions = actions, player = self.player)

        #Solange wie man Geduld hat wird ein baum erstellt, erweitert und Spiele zufällig zuende gespielt,
        #um am Ende den am meisten besuchten Knoten zurückzugeben
        for i in range(itermax):
            #Zur Wurzel zurückgehen und das Spiel zurück in den ausgangszustand bringen
            node = rootnode
            game = rootgame.Clone()

            #Phase 1: Selection. Sucht ein geeignetes Blatt mit dem UCB1 Algorithmus
            while node.untriedMoves == [] and node.childNodes != []:
                node = node.UCTSelectChild()
                #Bringt das Spiel in den jeweils zugehörigen Zustand
                game.execute_moves(node.move, game.get_opponent(node.playerJustMoved))

            #Phase 2: Expansion. Hängt den vom Modell als besten Zug bezeichneten Zug an den Baum an
            if node.untriedMoves != []:
                #Modell fragen was wir denn nehmen sollten!
                m = self.get_best_action(node.untriedMoves, game) 
                #Daten für neuen Knoten sammeln
                game.execute_moves(m, node.playerJustMoved)
                next_player = game.get_opponent(node.playerJustMoved)
                #Alle Würfelkombinationen und Züge besorgen
                next_actions = []
                rolls = [(i,j) for i in range(1,7) for j in range(i,7)]
                for (i,j) in rolls:
                    next_actions.extend(game.get_moves((i,j), next_player))
                #Knoten anhängen und direkt dort weitermachen
                node = node.AddChild(m, next_actions, next_player)

            #Phase 3: Simulation. Spielt das Spiel von dem gewählten Knoten aus zufällig zuende und ermittelt dem Gewinner
            winner = game.play_random(node.playerJustMoved)
            
            #Phase 4: Backpropagation. Update alle Vaterknoten bis hin zu Wurzel und erhöhe ggf. den Siegeszähler
            while node != None:
                #Siegeszähler deren Knoten erhöhen die den Sieger als Spieler eingetragen haben
                result = 1 if winner == node.playerJustMoved else 0
                node.Update(result)
                node = node.parentNode
        
        #Denjenigen Sohn von der Wurzel nehmen, der am meisten besucht wurde
        rootchilds = sorted(rootnode.childNodes, key = lambda c: c.visits)
        return rootchilds[-1].move
    
    #Normale bestimmung des besten Zuges wie der TDAgent
    def get_best_action(self, actions, game):
        v_best = 0
        a_best = None
        #Durchsucht alle gültigen, mögliche Züge nach dem Zug ab,
        #der für das Modell die höchste Siegwahrscheinlichkeit hat
        for a in actions:
            old_state = game.execute_moves(a, self.player)
            features = game.extractFeatures(self.player)
            v = self.model.get_output(features)
            v = 1. - v if self.player == game.players[0] else v
            if v > v_best:
                v_best = v
                a_best = a
            game.reset_to_state(old_state)
        return a_best

In [31]:
#TDAgent vs. TDMC Agent
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = OwnModel(sess, own_checkpoint_path, restore=True)
    model.test(enemyAgent=TDMCAgent('white', model), episodes=100, debug=False)

Restoring checkpoint: own_checkpoints/checkpoint 339707.ckpt-339707
INFO:tensorflow:Restoring parameters from own_checkpoints/checkpoint 339707.ckpt-339707
[Episode 0] TD-Gammon0.0 (black) vs TDMC (white) 1:0 of 1 games (100.00%)
[Episode 1] TD-Gammon0.0 (black) vs TDMC (white) 2:0 of 2 games (100.00%)
[Episode 2] TD-Gammon0.0 (black) vs TDMC (white) 3:0 of 3 games (100.00%)
[Episode 3] TD-Gammon0.0 (black) vs TDMC (white) 4:0 of 4 games (100.00%)
[Episode 4] TD-Gammon0.0 (black) vs TDMC (white) 5:0 of 5 games (100.00%)
[Episode 5] TD-Gammon0.0 (black) vs TDMC (white) 6:0 of 6 games (100.00%)
[Episode 6] TD-Gammon0.0 (black) vs TDMC (white) 7:0 of 7 games (100.00%)
[Episode 7] TD-Gammon0.0 (black) vs TDMC (white) 8:0 of 8 games (100.00%)
[Episode 8] TD-Gammon0.0 (black) vs TDMC (white) 9:0 of 9 games (100.00%)
[Episode 9] TD-Gammon0.0 (black) vs TDMC (white) 10:0 of 10 games (100.00%)
[Episode 10] TD-Gammon0.0 (black) vs TDMC (white) 11:0 of 11 games (100.00%)
[Episode 11] TD-Gammon0.0

MCTS entscheidet sich anscheinend meist anders als das Modell es vorschlägt. Daher wird je besser das Modell trainiert ist, der Monte Carlo Agent immer schlechter, da das Modell dann richtig liegt...

In [33]:
#TDAgent vs. TDAgent
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = OwnModel(sess, own_checkpoint_path, restore=True)
    model.test(enemyAgent=TDAgent2('white', model), episodes=200, debug=False)

Restoring checkpoint: own_checkpoints/checkpoint 1146457.ckpt-1146457
INFO:tensorflow:Restoring parameters from own_checkpoints/checkpoint 1146457.ckpt-1146457
[Episode 0] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 0:1 of 1 games (0.00%)
[Episode 1] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 1:1 of 2 games (50.00%)
[Episode 2] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 1:2 of 3 games (33.33%)
[Episode 3] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 2:2 of 4 games (50.00%)
[Episode 4] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 2:3 of 5 games (40.00%)
[Episode 5] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 2:4 of 6 games (33.33%)
[Episode 6] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 3:4 of 7 games (42.86%)
[Episode 7] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 3:5 of 8 games (37.50%)
[Episode 8] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 4:5 of 9 games (44.44%)
[Episode 9] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 5:5 of 10 games (50.00%)
[Episode 10] TD-Gammon0.0 (bla

[Episode 96] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 52:45 of 97 games (53.61%)
[Episode 97] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 53:45 of 98 games (54.08%)
[Episode 98] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 53:46 of 99 games (53.54%)
[Episode 99] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 53:47 of 100 games (53.00%)
[Episode 100] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 53:48 of 101 games (52.48%)
[Episode 101] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 54:48 of 102 games (52.94%)
[Episode 102] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 55:48 of 103 games (53.40%)
[Episode 103] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 55:49 of 104 games (52.88%)
[Episode 104] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 55:50 of 105 games (52.38%)
[Episode 105] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 55:51 of 106 games (51.89%)
[Episode 106] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 55:52 of 107 games (51.40%)
[Episode 107] TD-Gammon0.0 (black) vs TD-Gammon0.0

[Episode 191] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 104:88 of 192 games (54.17%)
[Episode 192] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 104:89 of 193 games (53.89%)
[Episode 193] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 105:89 of 194 games (54.12%)
[Episode 194] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 106:89 of 195 games (54.36%)
[Episode 195] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 106:90 of 196 games (54.08%)
[Episode 196] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 107:90 of 197 games (54.31%)
[Episode 197] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 107:91 of 198 games (54.04%)
[Episode 198] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 108:91 of 199 games (54.27%)
[Episode 199] TD-Gammon0.0 (black) vs TD-Gammon0.0 (white) 109:91 of 200 games (54.50%)


Der Modell spielt beide Seiten gleich stark.

In [25]:
"""
Profesor Harmelings Idee: Wieso eligibility traces und Gewichte anpassen, wenn das auch TF übernehmen kann?
"""

class TFAgent(object):

    def __init__(self, player, model):
        self.player = player
        self.model = model
        self.name = 'TFAgent'

    def get_action(self, actions, game):
        v_best = 0
        a_best = None
        #Durchsucht alle gültigen, mögliche Züge nach dem Zug ab,
        #der für das Modell die höchste Siegwahrscheinlichkeit hat
        for a in actions:
            old_state = game.execute_moves(a, self.player)
            features = game.extractFeatures(self.player)
            v = self.model.get_output(features)
            if v > v_best:
                v_best = v
                a_best = a
            game.reset_to_state(old_state)
        return a_best


class TFOptModel(object):
    def __init__(self, sess, checkpoint_path, restore=False):
        self.checkpoint_path = checkpoint_path + "/tfopt/"
        
        #session
        self.sess = sess
        self.global_step = tf.Variable(0, trainable=False, name='global_step')

        #lambda, leider ohne b da dies ein Schlüsselwort ist
        lamda = 0.7

        #learning rate 
        alpha = 0.1

        #Für leichteres abändern der Netzwerkarchitektur
        layer_size_input = 198
        layer_size_hidden = 40
        layer_size_output = 1

        # placeholders for input and target output
        self.x = tf.placeholder('float', [1, layer_size_input], name='x')
        self.V_next = tf.placeholder('float', [1, layer_size_output], name='V_next')

        #Zwei fully-connected, dense Layer: Ein Input-Layer (198 Units) und dann ein Hidden-Layer (40 Units)
        #Beide werden mit der sigmoid Funktion aktiviert
        prev_y = tf.layers.dense(self.x, layer_size_hidden, activation= tf.sigmoid)
        self.V = tf.layers.dense(prev_y, layer_size_output, activation= tf.sigmoid)
        
        # delta = V_next - V
        delta_op = tf.reduce_sum(self.V_next - self.V, name='delta')

        #Global_step wird bei jeden aufruf von sess.run um 1 erhöht
        global_step_op = self.global_step.assign_add(1)
        
        #Adam?
        opt_op = tf.train.AdamOptimizer().minimize(delta_op)
        
        #Alle Gradientenoperationen in einer train Operation zusammenfassen
        self.train_op = tf.group(global_step_op, opt_op, name='train')
        
        #Der Saver behält immer nur den neusten Checkpoint
        self.saver = tf.train.Saver(max_to_keep=1)

        #Variablen initialisieren 
        self.sess.run(tf.global_variables_initializer())

        #Modell wiederherstellen falls gewünscht
        if restore:
            self.restore()

    #Lädt den neusten (und auch einzigen) checkpoint 
    def restore(self):
        latest_checkpoint_path = tf.train.latest_checkpoint(self.checkpoint_path)
        if latest_checkpoint_path:
            print('Restoring checkpoint: {0}'.format(latest_checkpoint_path))
            self.saver.restore(self.sess, latest_checkpoint_path)

    #Erzeugt einen Outpt für die gegebenen features x
    def get_output(self, x):
        return self.sess.run(self.V, feed_dict={ self.x: x })

    #Testet das Modell gegen den angegebenen enemyAgent auf der neuen Backgammon Engine
    def test(self, enemyAgent=RandomAgent('white'), episodes=100, debug=False):
        players = [TFAgent('black', self), enemyAgent]
        
        winners = {'black':0, 'white':0}
        for episode in range(episodes):
            game = OwnGame()

            winner = game.play(players, debug=debug)
            winners[winner] += 1

            winners_total = sum(winners.values())
            print("[Episode %d] %s (%s) vs %s (%s) %d:%d of %d games (%.2f%%)" % (episode, \
                players[0].name, players[0].player, \
                players[1].name, players[1].player, \
                winners['black'], winners['white'], winners_total, \
                (winners['black'] / winners_total) * 100.0))

    def train(self, episodes, validation_interval, test_episodes=100):
        #Selbsttraining, Modell vs Modell
        players = [TFAgent('black', self), TFAgent('white', self)]

        for episode in range(episodes):
            #Immer wieder zwischendurch testen und den Fortschritt speichern
            if episode != 0 and episode % validation_interval == 0:
                self.saver.save(self.sess, self.checkpoint_path + 'checkpoint ' + str(global_step) + '.ckpt', global_step=global_step)
                print("Progress saved!")
                self.test(episodes=test_episodes)

            #Spiel initialisieren
            game = OwnGame()
            player_num = random.randint(0, 1)
            
            x = game.extractFeatures(players[player_num].player)

            #Spiel spielen bis es einen Sieger gibt
            game_step = 0
            while not game.get_winner():
                game.next_step(players[player_num], player_num)
                player_num = (player_num + 1) % 2

                #Das Modell mit jeden Schritt im Spiel trainieren
                x_next = game.extractFeatures(players[player_num].player)
                V_next = self.get_output(x_next)
                
                self.sess.run(self.train_op, feed_dict={ self.x: x, self.V_next: V_next })

                x = x_next
                game_step += 1

            #Gewinner ermitteln
            winner = 0 if game.get_winner() == game.players[0] else 1

            #Zu guter letzt reinforcement learning: Dem Modell noch eine "Belohnung" geben, wenn es gewonnen hat
            _, global_step = self.sess.run([
                self.train_op,
                self.global_step,
            ], feed_dict={ self.x: x, self.V_next: np.array([[winner]], dtype='float') })

            #Konsolenausgabe hübsch aufbereiten
            print("Game %d/%d (Winner: %s) in %d turns" % (episode, episodes, players[winner].player, game_step))
        #Am Ende noch mal speichern und 100 testen!
        self.saver.save(self.sess, self.checkpoint_path + 'checkpoint ' + str(global_step) + '.ckpt', global_step=global_step) 
        self.test(episodes=test_episodes)

In [26]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TFOptModel(sess, own_checkpoint_path, restore=False)
    model.train(5000,1000)

Game 0/5000 (Winner: black) in 105 turns
Game 1/5000 (Winner: black) in 53 turns
Game 2/5000 (Winner: black) in 68 turns
Game 3/5000 (Winner: black) in 68 turns
Game 4/5000 (Winner: white) in 49 turns
Game 5/5000 (Winner: black) in 59 turns
Game 6/5000 (Winner: white) in 70 turns
Game 7/5000 (Winner: white) in 55 turns
Game 8/5000 (Winner: white) in 73 turns
Game 9/5000 (Winner: white) in 64 turns
Game 10/5000 (Winner: black) in 76 turns
Game 11/5000 (Winner: white) in 77 turns
Game 12/5000 (Winner: black) in 52 turns
Game 13/5000 (Winner: black) in 69 turns
Game 14/5000 (Winner: white) in 53 turns
Game 15/5000 (Winner: black) in 68 turns
Game 16/5000 (Winner: black) in 49 turns
Game 17/5000 (Winner: black) in 76 turns
Game 18/5000 (Winner: black) in 88 turns
Game 19/5000 (Winner: black) in 65 turns
Game 20/5000 (Winner: black) in 67 turns
Game 21/5000 (Winner: black) in 62 turns
Game 22/5000 (Winner: black) in 53 turns
Game 23/5000 (Winner: black) in 53 turns
Game 24/5000 (Winner: bla

Game 197/5000 (Winner: white) in 68 turns
Game 198/5000 (Winner: black) in 103 turns
Game 199/5000 (Winner: black) in 194 turns
Game 200/5000 (Winner: white) in 100 turns
Game 201/5000 (Winner: black) in 52 turns
Game 202/5000 (Winner: white) in 53 turns
Game 203/5000 (Winner: black) in 151 turns
Game 204/5000 (Winner: white) in 145 turns
Game 205/5000 (Winner: white) in 136 turns
Game 206/5000 (Winner: white) in 80 turns
Game 207/5000 (Winner: white) in 146 turns
Game 208/5000 (Winner: white) in 59 turns
Game 209/5000 (Winner: white) in 67 turns
Game 210/5000 (Winner: white) in 77 turns
Game 211/5000 (Winner: black) in 65 turns
Game 212/5000 (Winner: black) in 86 turns
Game 213/5000 (Winner: black) in 134 turns
Game 214/5000 (Winner: black) in 139 turns
Game 215/5000 (Winner: black) in 73 turns
Game 216/5000 (Winner: black) in 119 turns
Game 217/5000 (Winner: white) in 51 turns
Game 218/5000 (Winner: white) in 96 turns
Game 219/5000 (Winner: white) in 149 turns
Game 220/5000 (Winner: 

Game 390/5000 (Winner: black) in 154 turns
Game 391/5000 (Winner: white) in 64 turns
Game 392/5000 (Winner: black) in 266 turns
Game 393/5000 (Winner: black) in 119 turns
Game 394/5000 (Winner: black) in 116 turns
Game 395/5000 (Winner: white) in 50 turns
Game 396/5000 (Winner: white) in 138 turns
Game 397/5000 (Winner: black) in 60 turns
Game 398/5000 (Winner: white) in 61 turns
Game 399/5000 (Winner: black) in 122 turns
Game 400/5000 (Winner: black) in 98 turns
Game 401/5000 (Winner: black) in 86 turns
Game 402/5000 (Winner: black) in 101 turns
Game 403/5000 (Winner: black) in 90 turns
Game 404/5000 (Winner: black) in 94 turns
Game 405/5000 (Winner: black) in 59 turns
Game 406/5000 (Winner: white) in 94 turns
Game 407/5000 (Winner: white) in 61 turns
Game 408/5000 (Winner: white) in 64 turns
Game 409/5000 (Winner: black) in 162 turns
Game 410/5000 (Winner: white) in 106 turns
Game 411/5000 (Winner: white) in 152 turns
Game 412/5000 (Winner: black) in 131 turns
Game 413/5000 (Winner: 

Game 583/5000 (Winner: black) in 155 turns
Game 584/5000 (Winner: white) in 111 turns
Game 585/5000 (Winner: black) in 97 turns
Game 586/5000 (Winner: black) in 96 turns
Game 587/5000 (Winner: white) in 99 turns
Game 588/5000 (Winner: black) in 133 turns
Game 589/5000 (Winner: white) in 143 turns
Game 590/5000 (Winner: black) in 92 turns
Game 591/5000 (Winner: black) in 109 turns
Game 592/5000 (Winner: black) in 138 turns
Game 593/5000 (Winner: black) in 129 turns
Game 594/5000 (Winner: black) in 145 turns
Game 595/5000 (Winner: black) in 175 turns
Game 596/5000 (Winner: black) in 272 turns
Game 597/5000 (Winner: black) in 88 turns
Game 598/5000 (Winner: black) in 82 turns
Game 599/5000 (Winner: white) in 139 turns
Game 600/5000 (Winner: black) in 109 turns
Game 601/5000 (Winner: white) in 94 turns
Game 602/5000 (Winner: white) in 146 turns
Game 603/5000 (Winner: white) in 63 turns
Game 604/5000 (Winner: white) in 138 turns
Game 605/5000 (Winner: black) in 89 turns
Game 606/5000 (Winne

KeyboardInterrupt: 

In [22]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = OwnModel(sess, own_checkpoint_path, restore=True)
    graph2 = tf.Graph()
    sess2 = tf.Session(graph=graph2)
    with sess2.as_default(), graph2.as_default():
        model2 = TFOptModel(sess2, own_checkpoint_path, restore=True)
        model.test(enemyAgent=TFAgent('white', model2), episodes=1000)

Restoring checkpoint: own_checkpoints/checkpoint 339707.ckpt-339707
INFO:tensorflow:Restoring parameters from own_checkpoints/checkpoint 339707.ckpt-339707
Restoring checkpoint: own_checkpoints//tfopt/checkpoint 559368.ckpt-559368
INFO:tensorflow:Restoring parameters from own_checkpoints//tfopt/checkpoint 559368.ckpt-559368
[Episode 0] TD-Gammon0.0 (black) vs TDTF (white) 1:0 of 1 games (100.00%)
[Episode 1] TD-Gammon0.0 (black) vs TDTF (white) 2:0 of 2 games (100.00%)
[Episode 2] TD-Gammon0.0 (black) vs TDTF (white) 3:0 of 3 games (100.00%)
[Episode 3] TD-Gammon0.0 (black) vs TDTF (white) 4:0 of 4 games (100.00%)
[Episode 4] TD-Gammon0.0 (black) vs TDTF (white) 5:0 of 5 games (100.00%)
[Episode 5] TD-Gammon0.0 (black) vs TDTF (white) 6:0 of 6 games (100.00%)
[Episode 6] TD-Gammon0.0 (black) vs TDTF (white) 7:0 of 7 games (100.00%)
[Episode 7] TD-Gammon0.0 (black) vs TDTF (white) 8:0 of 8 games (100.00%)
[Episode 8] TD-Gammon0.0 (black) vs TDTF (white) 9:0 of 9 games (100.00%)
[Episode

[Episode 103] TD-Gammon0.0 (black) vs TDTF (white) 103:1 of 104 games (99.04%)
[Episode 104] TD-Gammon0.0 (black) vs TDTF (white) 104:1 of 105 games (99.05%)
[Episode 105] TD-Gammon0.0 (black) vs TDTF (white) 105:1 of 106 games (99.06%)
[Episode 106] TD-Gammon0.0 (black) vs TDTF (white) 106:1 of 107 games (99.07%)
[Episode 107] TD-Gammon0.0 (black) vs TDTF (white) 107:1 of 108 games (99.07%)
[Episode 108] TD-Gammon0.0 (black) vs TDTF (white) 108:1 of 109 games (99.08%)
[Episode 109] TD-Gammon0.0 (black) vs TDTF (white) 109:1 of 110 games (99.09%)
[Episode 110] TD-Gammon0.0 (black) vs TDTF (white) 110:1 of 111 games (99.10%)
[Episode 111] TD-Gammon0.0 (black) vs TDTF (white) 111:1 of 112 games (99.11%)
[Episode 112] TD-Gammon0.0 (black) vs TDTF (white) 112:1 of 113 games (99.12%)
[Episode 113] TD-Gammon0.0 (black) vs TDTF (white) 113:1 of 114 games (99.12%)
[Episode 114] TD-Gammon0.0 (black) vs TDTF (white) 114:1 of 115 games (99.13%)
[Episode 115] TD-Gammon0.0 (black) vs TDTF (white) 1

[Episode 207] TD-Gammon0.0 (black) vs TDTF (white) 205:3 of 208 games (98.56%)
[Episode 208] TD-Gammon0.0 (black) vs TDTF (white) 206:3 of 209 games (98.56%)
[Episode 209] TD-Gammon0.0 (black) vs TDTF (white) 207:3 of 210 games (98.57%)
[Episode 210] TD-Gammon0.0 (black) vs TDTF (white) 208:3 of 211 games (98.58%)
[Episode 211] TD-Gammon0.0 (black) vs TDTF (white) 209:3 of 212 games (98.58%)
[Episode 212] TD-Gammon0.0 (black) vs TDTF (white) 210:3 of 213 games (98.59%)
[Episode 213] TD-Gammon0.0 (black) vs TDTF (white) 211:3 of 214 games (98.60%)
[Episode 214] TD-Gammon0.0 (black) vs TDTF (white) 212:3 of 215 games (98.60%)
[Episode 215] TD-Gammon0.0 (black) vs TDTF (white) 213:3 of 216 games (98.61%)
[Episode 216] TD-Gammon0.0 (black) vs TDTF (white) 214:3 of 217 games (98.62%)
[Episode 217] TD-Gammon0.0 (black) vs TDTF (white) 215:3 of 218 games (98.62%)
[Episode 218] TD-Gammon0.0 (black) vs TDTF (white) 216:3 of 219 games (98.63%)
[Episode 219] TD-Gammon0.0 (black) vs TDTF (white) 2

[Episode 311] TD-Gammon0.0 (black) vs TDTF (white) 307:5 of 312 games (98.40%)
[Episode 312] TD-Gammon0.0 (black) vs TDTF (white) 308:5 of 313 games (98.40%)
[Episode 313] TD-Gammon0.0 (black) vs TDTF (white) 309:5 of 314 games (98.41%)
[Episode 314] TD-Gammon0.0 (black) vs TDTF (white) 310:5 of 315 games (98.41%)
[Episode 315] TD-Gammon0.0 (black) vs TDTF (white) 311:5 of 316 games (98.42%)
[Episode 316] TD-Gammon0.0 (black) vs TDTF (white) 312:5 of 317 games (98.42%)
[Episode 317] TD-Gammon0.0 (black) vs TDTF (white) 313:5 of 318 games (98.43%)
[Episode 318] TD-Gammon0.0 (black) vs TDTF (white) 314:5 of 319 games (98.43%)
[Episode 319] TD-Gammon0.0 (black) vs TDTF (white) 315:5 of 320 games (98.44%)
[Episode 320] TD-Gammon0.0 (black) vs TDTF (white) 316:5 of 321 games (98.44%)
[Episode 321] TD-Gammon0.0 (black) vs TDTF (white) 317:5 of 322 games (98.45%)
[Episode 322] TD-Gammon0.0 (black) vs TDTF (white) 318:5 of 323 games (98.45%)
[Episode 323] TD-Gammon0.0 (black) vs TDTF (white) 3

[Episode 415] TD-Gammon0.0 (black) vs TDTF (white) 411:5 of 416 games (98.80%)
[Episode 416] TD-Gammon0.0 (black) vs TDTF (white) 411:6 of 417 games (98.56%)
[Episode 417] TD-Gammon0.0 (black) vs TDTF (white) 412:6 of 418 games (98.56%)
[Episode 418] TD-Gammon0.0 (black) vs TDTF (white) 413:6 of 419 games (98.57%)
[Episode 419] TD-Gammon0.0 (black) vs TDTF (white) 413:7 of 420 games (98.33%)
[Episode 420] TD-Gammon0.0 (black) vs TDTF (white) 414:7 of 421 games (98.34%)
[Episode 421] TD-Gammon0.0 (black) vs TDTF (white) 415:7 of 422 games (98.34%)
[Episode 422] TD-Gammon0.0 (black) vs TDTF (white) 416:7 of 423 games (98.35%)
[Episode 423] TD-Gammon0.0 (black) vs TDTF (white) 417:7 of 424 games (98.35%)
[Episode 424] TD-Gammon0.0 (black) vs TDTF (white) 418:7 of 425 games (98.35%)
[Episode 425] TD-Gammon0.0 (black) vs TDTF (white) 419:7 of 426 games (98.36%)
[Episode 426] TD-Gammon0.0 (black) vs TDTF (white) 420:7 of 427 games (98.36%)
[Episode 427] TD-Gammon0.0 (black) vs TDTF (white) 4

[Episode 519] TD-Gammon0.0 (black) vs TDTF (white) 512:8 of 520 games (98.46%)
[Episode 520] TD-Gammon0.0 (black) vs TDTF (white) 513:8 of 521 games (98.46%)
[Episode 521] TD-Gammon0.0 (black) vs TDTF (white) 514:8 of 522 games (98.47%)
[Episode 522] TD-Gammon0.0 (black) vs TDTF (white) 515:8 of 523 games (98.47%)
[Episode 523] TD-Gammon0.0 (black) vs TDTF (white) 516:8 of 524 games (98.47%)
[Episode 524] TD-Gammon0.0 (black) vs TDTF (white) 517:8 of 525 games (98.48%)
[Episode 525] TD-Gammon0.0 (black) vs TDTF (white) 518:8 of 526 games (98.48%)
[Episode 526] TD-Gammon0.0 (black) vs TDTF (white) 519:8 of 527 games (98.48%)
[Episode 527] TD-Gammon0.0 (black) vs TDTF (white) 520:8 of 528 games (98.48%)
[Episode 528] TD-Gammon0.0 (black) vs TDTF (white) 521:8 of 529 games (98.49%)
[Episode 529] TD-Gammon0.0 (black) vs TDTF (white) 522:8 of 530 games (98.49%)
[Episode 530] TD-Gammon0.0 (black) vs TDTF (white) 523:8 of 531 games (98.49%)
[Episode 531] TD-Gammon0.0 (black) vs TDTF (white) 5

[Episode 623] TD-Gammon0.0 (black) vs TDTF (white) 615:9 of 624 games (98.56%)
[Episode 624] TD-Gammon0.0 (black) vs TDTF (white) 616:9 of 625 games (98.56%)
[Episode 625] TD-Gammon0.0 (black) vs TDTF (white) 617:9 of 626 games (98.56%)
[Episode 626] TD-Gammon0.0 (black) vs TDTF (white) 618:9 of 627 games (98.56%)
[Episode 627] TD-Gammon0.0 (black) vs TDTF (white) 619:9 of 628 games (98.57%)
[Episode 628] TD-Gammon0.0 (black) vs TDTF (white) 620:9 of 629 games (98.57%)
[Episode 629] TD-Gammon0.0 (black) vs TDTF (white) 621:9 of 630 games (98.57%)
[Episode 630] TD-Gammon0.0 (black) vs TDTF (white) 622:9 of 631 games (98.57%)
[Episode 631] TD-Gammon0.0 (black) vs TDTF (white) 623:9 of 632 games (98.58%)
[Episode 632] TD-Gammon0.0 (black) vs TDTF (white) 623:10 of 633 games (98.42%)
[Episode 633] TD-Gammon0.0 (black) vs TDTF (white) 624:10 of 634 games (98.42%)
[Episode 634] TD-Gammon0.0 (black) vs TDTF (white) 625:10 of 635 games (98.43%)
[Episode 635] TD-Gammon0.0 (black) vs TDTF (white

[Episode 828] TD-Gammon0.0 (black) vs TDTF (white) 819:10 of 829 games (98.79%)
[Episode 829] TD-Gammon0.0 (black) vs TDTF (white) 820:10 of 830 games (98.80%)
[Episode 830] TD-Gammon0.0 (black) vs TDTF (white) 821:10 of 831 games (98.80%)
[Episode 831] TD-Gammon0.0 (black) vs TDTF (white) 822:10 of 832 games (98.80%)
[Episode 832] TD-Gammon0.0 (black) vs TDTF (white) 823:10 of 833 games (98.80%)
[Episode 833] TD-Gammon0.0 (black) vs TDTF (white) 824:10 of 834 games (98.80%)
[Episode 834] TD-Gammon0.0 (black) vs TDTF (white) 825:10 of 835 games (98.80%)
[Episode 835] TD-Gammon0.0 (black) vs TDTF (white) 826:10 of 836 games (98.80%)
[Episode 836] TD-Gammon0.0 (black) vs TDTF (white) 827:10 of 837 games (98.81%)
[Episode 837] TD-Gammon0.0 (black) vs TDTF (white) 828:10 of 838 games (98.81%)
[Episode 838] TD-Gammon0.0 (black) vs TDTF (white) 829:10 of 839 games (98.81%)
[Episode 839] TD-Gammon0.0 (black) vs TDTF (white) 830:10 of 840 games (98.81%)
[Episode 840] TD-Gammon0.0 (black) vs TD

[Episode 931] TD-Gammon0.0 (black) vs TDTF (white) 922:10 of 932 games (98.93%)
[Episode 932] TD-Gammon0.0 (black) vs TDTF (white) 923:10 of 933 games (98.93%)
[Episode 933] TD-Gammon0.0 (black) vs TDTF (white) 924:10 of 934 games (98.93%)
[Episode 934] TD-Gammon0.0 (black) vs TDTF (white) 925:10 of 935 games (98.93%)
[Episode 935] TD-Gammon0.0 (black) vs TDTF (white) 926:10 of 936 games (98.93%)
[Episode 936] TD-Gammon0.0 (black) vs TDTF (white) 927:10 of 937 games (98.93%)
[Episode 937] TD-Gammon0.0 (black) vs TDTF (white) 928:10 of 938 games (98.93%)
[Episode 938] TD-Gammon0.0 (black) vs TDTF (white) 929:10 of 939 games (98.94%)
[Episode 939] TD-Gammon0.0 (black) vs TDTF (white) 930:10 of 940 games (98.94%)
[Episode 940] TD-Gammon0.0 (black) vs TDTF (white) 931:10 of 941 games (98.94%)
[Episode 941] TD-Gammon0.0 (black) vs TDTF (white) 932:10 of 942 games (98.94%)
[Episode 942] TD-Gammon0.0 (black) vs TDTF (white) 933:10 of 943 games (98.94%)
[Episode 943] TD-Gammon0.0 (black) vs TD

Der Optimierer minmiert zwar das Delta aber gibt dem Modell keine Richtung bzw. Bewertung für gute/schlechte Züge, sodass das Modell nicht lernt was gut oder schlecht ist und somit komplett random bewertet, was sich auch in den Tests widerspiegelt.

In [48]:
from abc import ABC, abstractmethod
from Player import RandomPlayer, ModelPlayer
from CythonBackgammon import Game
import tensorflow as tf
import numpy as np
import random
import os

class NeuralNetModel(ABC):
    
    def __init__(self, sess, input_size = 198, hidden_size = 40, output_size = 1, name = None, restore=False):
    
        if not name:
            name = self.get_name()
    
        #Speicherort ermitteln
        self.checkpoint_path = os.environ.get('CHECKPOINT_PATH', 'checkpoints/' + name +'/')
        
        #session
        self.sess = sess
        self.global_step = tf.Variable(0, trainable=False, name='global_step')

        # placeholders for input and target output
        self.x = tf.placeholder('float', [1, input_size], name='x')
        self.V_next = tf.placeholder('float', [1, output_size], name='V_next')

        #Zwei fully-connected, dense Layer: Ein Input-Layer (198 Units) und dann ein Hidden-Layer (40 Units)
        #Beide werden mit der sigmoid Funktion aktiviert
        prev_y = tf.layers.dense(self.x, hidden_size, activation= tf.sigmoid)
        self.V = tf.layers.dense(prev_y, output_size, activation= tf.sigmoid)
        
        # delta = V_next - V
        self.delta_op = tf.reduce_sum(self.V_next - self.V, name='delta')

        #Erstellt eine Trainings- Operation und speichert diese ab
        self.train_op = self.create_training_op()

        #Der Saver behält immer nur den neusten Checkpoint
        self.saver = tf.train.Saver(max_to_keep=1)

        #Variablen initialisieren 
        self.sess.run(tf.global_variables_initializer())

        #Modell wiederherstellen falls gewünscht
        if restore:
            self.restore()

    @abstractmethod
    def create_training_op(self):
        pass
    
    @abstractmethod
    def get_name(self):
        pass
    
    """
        Bekannte Methoden aus der TD-GammonModel Klasse (004)
    """
            
        #Lädt den neusten (und auch einzigen) checkpoint 
    def restore(self):
        latest_checkpoint_path = tf.train.latest_checkpoint(self.checkpoint_path)
        if latest_checkpoint_path:
            print("Restoring checkpoint: {0}".format(latest_checkpoint_path))
            self.saver.restore(self.sess, latest_checkpoint_path)

    #Erzeugt einen Outpt für die gegebenen features x
    def get_output(self, x):
        return self.sess.run(self.V, feed_dict={ self.x: x })

    #Testet das Modell gegen den angegebenen enemyAgent
    def test(self, enemyPlayer=RandomPlayer('white'), games=100, debug=False):
        players = [ModelPlayer('black', self), enemyPlayer]
        
        winners = {'black':0, 'white':0}
        for i in range(games):
            game = Game()

            winner = game.play(players, debug=debug)
            winners[winner] += 1

            winners_total = sum(winners.values())
            print("[Game %d] %s (%s) vs %s (%s) %d:%d of %d games (%.2f%%)" % (i, \
                players[0].get_name(), players[0].player, \
                players[1].get_name(), players[1].player, \
                winners['black'], winners['white'], winners_total, \
                (winners['black'] / winners_total) * 100.0))
            
    def train(self, games, validation_interval, test_games=100):
        #Selbsttraining, Modell vs Modell
        players = [ModelPlayer('black', self), ModelPlayer('white', self)]

        for i in range(games):
            #Immer wieder zwischendurch testen und den Fortschritt speichern
            if i != 0 and i % validation_interval == 0:
                self.saver.save(self.sess, self.checkpoint_path + 'checkpoint.ckpt', global_step=global_step)
                print("Progress saved!")
                self.test(games = test_games)

            #Spiel initialisieren
            game = Game()
            player_num = random.randint(0, 1)
            
            x = game.extractFeatures(players[player_num].player)

            #Spiel spielen bis es einen Sieger gibt
            game_step = 0
            while not game.get_winner():
                game.next_step(players[player_num], player_num)
                player_num = (player_num + 1) % 2

                #Das Modell mit jeden Schritt im Spiel trainieren
                x_next = game.extractFeatures(players[player_num].player)
                V_next = self.get_output(x_next)
                
                self.sess.run([self.train_op, self.delta_op], feed_dict={ self.x: x, self.V_next: V_next })
                
                
                
                x = x_next
                game_step += 1

            #Gewinner ermitteln
            winner = 0 if game.get_winner() == game.players[0] else 1

            #Zu guter letzt reinforcement learning: Dem Modell noch eine "Belohnung" geben, wenn es gewonnen hat
            _, global_step, res = self.sess.run([
                self.train_op,
                self.global_step,
                self.grads, 
            ], feed_dict={ self.x: x, self.V_next: np.array([[winner]], dtype='float') })
            
            for j in range(len(res)):
                    print(self.grads[j], res[j])

            #Konsolenausgabe hübsch aufbereiten
            print("Game %d/%d (Winner: %s) in %d turns" % (i, games, players[winner].player, game_step))
            
        #Am Ende noch mal speichern und 100 testen!
        self.saver.save(self.sess, self.checkpoint_path + 'checkpoint.ckpt', global_step=global_step) 
        self.test(games = test_games)
		
		
"""
	Implementationen dieser Abstrakten Klasse
"""

class TDGammonModel(NeuralNetModel):

	def create_training_op(self):
		#lambda, leider ohne b da dies ein Schlüsselwort ist
		lamda = tf.constant(0.7)

		#learning rate 
		alpha = tf.constant(0.1)
	
		#Global_step wird bei jeden aufruf von sess.run um 1 erhöht
		global_step_op = self.global_step.assign_add(1)

		# get gradients of output V wrt trainable variables (weights and biases)
		tvars = tf.trainable_variables()
		grads = tf.gradients(self.V, tvars)

		#Alle Variablen werden angepasst und mittels eligebility traces wird er Wert
		#der einzelnen Gewichte angepasst um gute bzw. schlechte Entscheidungen zu reflektieren
		apply_gradients = []

		self.grads = []
        
		for grad, var in zip(grads, tvars):
			# e-> = lambda * e-> + <grad of output w.r.t weights>
			trace = tf.Variable(tf.zeros(grad.get_shape()), trainable=False, name='trace')
			trace_op = trace.assign((lamda * trace) + grad)
            
			self.grads.append(grad)
			print(grad)
			# grad with trace = alpha * delta * e
			grad_trace = alpha * self.delta_op * trace_op

			grad_apply = var.assign_add(grad_trace)
			apply_gradients.append(grad_apply)
			
		#Den global_step mitzählen lassen
		apply_gradients.append(global_step_op)

		#Alle Gradientenoperationen in einer train Operation zusammenfassen
		return tf.group(*apply_gradients, name='train')
			
	def get_name(self):
		return "TD-Gammon"

In [49]:
import tensorflow as tf

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, hidden_size = 50, name = "tracetest", restore=False)
    print(model.grads)
    model.train(10, 100)

Tensor("gradients/dense/MatMul_grad/MatMul_1:0", shape=(198, 50), dtype=float32)
Tensor("gradients/dense/BiasAdd_grad/BiasAddGrad:0", shape=(50,), dtype=float32)
Tensor("gradients/dense_1/MatMul_grad/MatMul_1:0", shape=(50, 1), dtype=float32)
Tensor("gradients/dense_1/BiasAdd_grad/BiasAddGrad:0", shape=(1,), dtype=float32)
[<tf.Tensor 'gradients/dense/MatMul_grad/MatMul_1:0' shape=(198, 50) dtype=float32>, <tf.Tensor 'gradients/dense/BiasAdd_grad/BiasAddGrad:0' shape=(50,) dtype=float32>, <tf.Tensor 'gradients/dense_1/MatMul_grad/MatMul_1:0' shape=(50, 1) dtype=float32>, <tf.Tensor 'gradients/dense_1/BiasAdd_grad/BiasAddGrad:0' shape=(1,) dtype=float32>]
Tensor("gradients/dense/MatMul_grad/MatMul_1:0", shape=(198, 50), dtype=float32) [[ 0.0060851   0.00937578 -0.01012516 ... -0.00952229 -0.01669281
  -0.01688859]
 [ 0.0060851   0.00937578 -0.01012516 ... -0.00952229 -0.01669281
  -0.01688859]
 [ 0.0060851   0.00937578 -0.01012516 ... -0.00952229 -0.01669281
  -0.01688859]
 ...
 [ 0.006

Tensor("gradients/dense/MatMul_grad/MatMul_1:0", shape=(198, 50), dtype=float32) [[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.00107414  0.00205917 -0.00374352 ... -0.00347582 -0.00570454
  -0.00557218]
 [ 0.00201402  0.00386095 -0.0070191  ... -0.00651716 -0.01069602
  -0.01044784]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
Tensor("gradients/dense/BiasAdd_grad/BiasAddGrad:0", shape=(50,), dtype=float32) [ 0.00201402  0.00386095 -0.0070191   0.00433035 -0.01190972  0.00537704
  0.00041664 -0.00574962 -0.0053115  -0.00182137  0.00049409 -0.00267921
  0.00067697 -0.00182248  0.00209224  0.00219461 -0.00256072  0.00858123
 -0.00188529 -0.00644113 -0.00803601  0.00370683 -0.00397565  0.00359118
 -0.00993255 -0.00265581  0.00344404 -0.0017893  -0.00239307 -0.01216365
  0.00960434 

Tensor("gradients/dense/MatMul_grad/MatMul_1:0", shape=(198, 50), dtype=float32) [[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.00101092  0.00187246 -0.00350352 ... -0.00343076 -0.00530181
  -0.0053016 ]
 [ 0.00189548  0.00351087 -0.0065691  ... -0.00643267 -0.0099409
  -0.00994049]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
Tensor("gradients/dense/BiasAdd_grad/BiasAddGrad:0", shape=(50,), dtype=float32) [ 0.00189548  0.00351087 -0.0065691   0.00398143 -0.01125931  0.00511941
  0.00043363 -0.00572272 -0.00514432 -0.00175141  0.0004274  -0.00263513
  0.0005695  -0.00170155  0.00174479  0.00193364 -0.00249305  0.00827704
 -0.00173806 -0.00607812 -0.00812422  0.0033443  -0.0039397   0.00339301
 -0.00958559 -0.00262727  0.00328469 -0.00188957 -0.00221606 -0.01166977
  0.00921599 -

[Game 44] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 35:10 of 45 games (77.78%)
[Game 45] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 36:10 of 46 games (78.26%)
[Game 46] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 37:10 of 47 games (78.72%)
[Game 47] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 37:11 of 48 games (77.08%)
[Game 48] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 38:11 of 49 games (77.55%)
[Game 49] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 39:11 of 50 games (78.00%)
[Game 50] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 40:11 of 51 games (78.43%)
[Game 51] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 41:11 of 52 games (78.85%)
[Game 52] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 42:11 of 53 games (79.25%)
[Game 53] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 43:11 of 54 games (79.63%)
[Game 54] ModelPlayer [TD-Gammon] (black) vs RandomPlayer (white) 44:1